# 一、数据准备

## 1.建立数据库

In [ ]:
import sqlite3
import pandas as pd

# 创建数据库连接
conn = sqlite3.connect('nomis_data.db')

# TSV文件和对应的表名
tsv_files = {
    # 'D:/Dissertation/project/data/observational data/1annual population survey/SOC2010_21370851878959376_data.tsv': 'SOC2010_employmentpercent',
    # 'D:/Dissertation/project/data/observational data/1annual population survey/SOC2020_21354692132373544_data.tsv': 'SOC2020_employmentpercent',
    # 'D:/Dissertation/project/data/observational data/2annual population survey/21363951043487306_data.tsv': 'un_employment_qualification_byage',
    # 'D:/Dissertation/project/data/observational data/3annual survey of hours and earnings - workplace analysis/5331161679142277_data.tsv': 'pay_time_weekly',
    'D:/Dissertation/project/data/observational data/4RM011 - Country of birth by age/23825931246572534_data.tsv': 'population_byage',
    # 'D:/Dissertation/project/data/observational data/5RM016 - Distance travelled to work by occupation/21386281053368857_data.tsv': 'commute_byoccupation',
    # 'D:/Dissertation/project/data/observational data/6TS060 - Industry/23831791361490009_data.tsv':'population_industry',
    # 'D:/Dissertation/project/data/observational data/1annual population survey/SOC2010_sub_640423263229051_data.tsv':'SOC2010_sub',
    # 'D:/Dissertation/project/data/observational data/1annual population survey/SOC2020_sub_22371061404360831.tsv':'SOC2020_sub'
}


for tsv_file, table_name in tsv_files.items():
    try:
        # 读取TSV文件
        df = pd.read_csv(tsv_file, sep='\t')
        
        # 导入到SQLite
        df.to_sql(table_name, conn, if_exists='replace', index=False)
        
        print(f"成功导入 {len(df)} 行到表 {table_name}")
    except Exception as e:
        print(f"处理文件 {tsv_file} 时出错: {e}")

conn.close()

## 2.数据清洗-修改表格式

在SOC1020——combined表中添加London---E09000001数据

In [ ]:
import sqlite3
import pandas as pd

# 步骤1：定义输入数据
data = {
    'geogcode': ['E09000001'] * 108,  # 9 variables * 12 years
    'variable': [str(v) for v in range(1, 10)] * 12,
    'year': [y for y in range(2013, 2025) for _ in range(9)],
    'value': [
        11.4, 24.9, 17.5, 10.4, 7.7, 7.7, 6.8, 4.0, 8.8,  # 2013
        11.8, 23.7, 18.0, 10.6, 7.5, 7.4, 6.7, 4.2, 9.3,  # 2014
        11.7, 24.0, 17.2, 10.0, 7.7, 8.1, 6.8, 4.4, 9.7,  # 2015
        12.4, 25.2, 17.9, 9.2, 7.0, 7.5, 6.7, 4.7, 9.0,   # 2016
        12.4, 25.6, 17.4, 9.7, 7.5, 7.1, 6.7, 4.4, 8.8,   # 2017
        12.3, 26.2, 18.5, 9.5, 6.9, 7.2, 5.9, 4.4, 8.5,   # 2018
        13.5, 26.5, 18.3, 8.7, 7.0, 7.1, 5.8, 4.5, 8.0,   # 2019
        13.3, 29.8, 19.2, 9.3, 6.0, 6.7, 5.3, 3.4, 6.4,   # 2020
        12.6, 30.5, 18.5, 10.0, 5.2, 7.1, 5.5, 3.2, 6.8,  # 2021
        12.8, 33.6, 16.2, 8.9, 6.3, 7.0, 5.0, 3.2, 6.5,   # 2022
        13.1, 33.2, 16.9, 8.9, 6.0, 6.5, 5.5, 3.2, 6.6,   # 2023
        13.9, 33.7, 17.1, 8.0, 5.5, 6.9, 4.7, 3.3, 6.7    # 2024
    ]
}
df = pd.DataFrame(data)

# 步骤2：连接数据库
conn = sqlite3.connect('nomis_data.db')
cursor = conn.cursor()

# 创建表（如果不存在）
cursor.execute("""
CREATE TABLE IF NOT EXISTS SOC1020_combined (
    geogcode TEXT,
    year INTEGER,
    variable TEXT,
    value REAL,
    PRIMARY KEY (geogcode, year, variable)
)
""")

# 步骤3：删除geogcode='E09000001'的现有记录
cursor.execute("DELETE FROM SOC1020_combined WHERE geogcode = 'E09000001'")

# 步骤4：插入新数据
df.to_sql('SOC1020_combined', conn, if_exists='append', index=False)

# 步骤5：提交更改并关闭连接
conn.commit()
conn.close()

print("已删除E09000001的原有记录并添加新数据到SOC1020_combined表")

### 对SOC1020_combined进行插值补充

In [ ]:
import sqlite3
import pandas as pd
import numpy as np

# 连接数据库
conn = sqlite3.connect('nomis_data.db')
cursor = conn.cursor()

# 步骤1：读取SOC1020_combined表数据
query = """
SELECT geogcode, year, variable, value
FROM SOC1020_combined
"""
df = pd.read_sql_query(query, conn)

# 步骤2：定义插值和误差计算函数
def interpolate_and_compute_error(group):
    # 提取年份和值
    years = group['year'].values
    values = group['value'].values
    
    # 标记空值或0值的位置
    mask = (pd.isna(values) | (values == 0))
    
    # 初始化MSE
    mse = 0.0
    
    if mask.all():
        # 如果全为0或空值，填充为0，MSE为0
        group['value'] = group['value'].fillna(0)
        return group, mse
    
    # 获取非空、非0的数据点
    valid_mask = ~mask
    valid_years = years[valid_mask]
    valid_values = values[valid_mask]
    
    if len(valid_values) >= 3:
        # 拟合2阶多项式
        try:
            coeffs = np.polyfit(valid_years, valid_values, deg=2)
            poly = np.poly1d(coeffs)
            # 对空值或0值插值
            group.loc[mask, 'value'] = poly(years[mask])
            # 严格限制在[0, 60]
            group['value'] = np.clip(group['value'], 0, 60)
            # 计算MSE（仅针对有效点）
            predicted_values = np.clip(poly(valid_years), 0, 60)
            mse = np.mean((valid_values - predicted_values) ** 2)
        except np.linalg.LinAlgError:
            # 如果多项式拟合失败，切换到线性插值
            group['value'] = group['value'].interpolate(method='linear', limit_direction='both')
            group['value'] = np.clip(group['value'], 0, 60)
            group['value'] = group['value'].fillna(group['value'].mean())
            group['value'] = np.clip(group['value'], 0, 60)
            if len(valid_values) > 0:
                valid_indices = group.index[valid_mask]
                predicted_values = group.loc[valid_indices, 'value']
                mse = np.mean((valid_values - predicted_values) ** 2)
    else:
        # 使用线性插值
        group['value'] = group['value'].interpolate(method='linear', limit_direction='both')
        # 填充剩余NaN
        group['value'] = group['value'].fillna(group['value'].mean())
        # 严格限制在[0, 60]
        group['value'] = np.clip(group['value'], 0, 60)
        # 如果有有效点，计算MSE
        if len(valid_values) > 0:
            valid_indices = group.index[valid_mask]
            predicted_values = group.loc[valid_indices, 'value']
            mse = np.mean((valid_values - predicted_values) ** 2)
    
    # 额外检查负值
    if (group['value'] < 0).any():
        print(f"警告：组(geogcode={group['geogcode'].iloc[0]}, variable={group['variable'].iloc[0]}) "
              f"包含负值，已强制设为0")
        group['value'] = np.clip(group['value'], 0, 60)
    
    return group, mse

# 步骤3：按geogcode和variable分组，应用插值并收集误差
results = []
errors = []
for (geogcode, variable), group in df.groupby(['geogcode', 'variable']):
    interpolated_group, mse = interpolate_and_compute_error(group.copy())
    results.append(interpolated_group)
    errors.append({'geogcode': geogcode, 'variable': variable, 'mse': mse})

# 合并插值结果
interpolated_df = pd.concat(results).reset_index(drop=True)
# 创建误差DataFrame
errors_df = pd.DataFrame(errors)

# 步骤4：检查负值
if (interpolated_df['value'] < 0).any():
    print("警告：插值结果中仍包含负值，已强制设为0")
    interpolated_df['value'] = np.clip(interpolated_df['value'], 0, 60)

# 步骤5：保存插值结果到SOC1020_combined_interpolated表
cursor.execute("DROP TABLE IF EXISTS SOC1020_combined_interpolated")
interpolated_df.to_sql('SOC1020_combined_interpolated', conn, index=False, if_exists='replace')

# 步骤7：打印误差结果
print("插值误差（均方误差，MSE）：")
print(errors_df)

# 步骤8：提交更改并关闭连接
conn.commit()
conn.close()

print("/n插值完成，结果已保存到SOC1020_combined_interpolated表")
print("误差已保存到interpolation_errors表")

E06000053和E09000001缺失数据太多，不再进行后续分析

### soc2010_sub和soc2020_sub合并

In [ ]:
import pandas as pd
import sqlite3

# 连接数据库并读取数据
conn = sqlite3.connect("nomis_data.db")
soc2010 = pd.read_sql("SELECT * FROM SOC2010_sub_cleaned", conn)
soc2020 = pd.read_sql("SELECT * FROM SOC2020_sub_cleaned", conn)

# 合并两个表，SOC2010保持不变
soc1020 = pd.concat([soc2010, soc2020], ignore_index=True)

# 数据清洗：确保year列是整数类型，value列是数值类型
soc1020['year'] = pd.to_numeric(soc1020['year'], errors='coerce').fillna(0).astype(int)
soc1020['value'] = pd.to_numeric(soc1020['value'], errors='coerce').fillna(0)

# 对SOC2020数据进行修改（只修改2020及以后的数据）
soc2020_data = soc1020[soc1020['year'] >= 2020].copy()

# 按geogcode、variable和year分组
grouped = soc2020_data.groupby(['geogcode', 'variable', 'year'])

def safe_float_conversion(x):
    """安全转换为float的函数"""
    try:
        return float(x)
    except (ValueError, TypeError):
        return 0.0

def get_value(df, geogcode, variable, year):
    """安全获取特定条件的value值"""
    filtered = df[(df['geogcode'] == geogcode) & 
                 (df['variable'] == variable) &
                 (df['year'] == year)]
    return safe_float_conversion(filtered['value'].iloc[0]) if not filtered.empty else 0.0

def modify_values(group):
    """修改值的函数，增加了错误处理"""
    try:
        geogcode, variable, year = group.name
        
        # 获取当前组的值
        current_val = safe_float_conversion(group['value'].iloc[0]) if not group.empty else 0.0
        
        # 根据variable类型应用不同的修改公式
        if variable == 12:
            val_62 = get_value(soc2020_data, geogcode, 62, year)
            val_71 = get_value(soc2020_data, geogcode, 71, year)
            new_value = current_val * 0.82 + val_62 * 0.01 + val_71 * 0.17
            
        elif variable == 32:
            val_22 = get_value(soc2020_data, geogcode, 22, year)
            val_61 = get_value(soc2020_data, geogcode, 61, year)
            new_value = val_22 * 0.07 + current_val * 0.92 + val_61 * 0.01
            
        elif variable == 33:
            val_63 = get_value(soc2020_data, geogcode, 63, year)
            new_value = current_val * 0.94 + val_63 * 0.06
            
        elif variable == 34:
            val_21 = get_value(soc2020_data, geogcode, 21, year)
            new_value = current_val * 0.86 + val_21 * 0.14
            
        elif variable == 35:
            val_24 = get_value(soc2020_data, geogcode, 24, year)
            new_value = current_val * 0.85 + val_24 * 0.15
            
        elif variable == 42:
            val_41 = get_value(soc2020_data, geogcode, 41, year)
            val_31 = get_value(soc2020_data, geogcode, 31, year)
            new_value = current_val * 0.83 + val_41 * 0.15 + val_31 * 0.02
            
        elif variable == 61:
            val_32 = get_value(soc2020_data, geogcode, 32, year)
            new_value = current_val * 0.95 + val_32 * 0.05
            
        elif variable == 72:
            val_41 = get_value(soc2020_data, geogcode, 41, year)
            new_value = current_val * 0.85 + val_41 * 0.15
            
        else:
            new_value = current_val  # 不满足条件的变量保持原值
            
        group['value'] = new_value
        
    except Exception as e:
        print(f"处理组 {group.name} 时出错: {str(e)}")
        group['value'] = 0.0  # 出错时设为默认值
        
    return group

# 应用修改函数
modified_soc2020 = grouped.apply(modify_values).reset_index(drop=True)

# 将修改后的SOC2020数据与未修改的SOC2010数据合并
soc2010_data = soc1020[soc1020['year'] < 2020]
final_soc1020 = pd.concat([soc2010_data, modified_soc2020], ignore_index=True)
final_soc1020.to_sql('SOC1020_sub_combined', conn, if_exists='replace', index=False)

# 保存结果
final_soc1020.to_csv('SOC1020_sub.csv', index=False)

### 对SOC1020_sub_combined插值补充

In [ ]:
import sqlite3
# 连接数据库
conn = sqlite3.connect("nomis_data.db")
cursor = conn.cursor()
cursor.execute("DROP TABLE population_industy")
conn.close()

(按照2013-2024年统计数据进行多项式插值补充)

In [ ]:
import sqlite3
import pandas as pd
import numpy as np

# 连接到 SQLite 数据库
conn = sqlite3.connect('nomis_data.db')

# 加载 SOC1020_sub_combined 表
df = pd.read_sql_query("SELECT * FROM SOC1020_sub_combined", conn)

# 确保 'value' 列为数值型，非数值转为 NaN
df['value'] = pd.to_numeric(df['value'], errors='coerce')

# 第一步：对 2024 年的空值进行多项式插值
# 创建插值表的副本
df_interpolated = df.copy()

# 按 geogcode 和 variable 分组，进行多项式插值
def polynomial_interpolate_group(group, degree=2):
    # 按年份排序，确保时间顺序
    group = group.sort_values('year')
    # 去除 NaN 值以拟合多项式
    valid = group.dropna(subset=['value'])
    if len(valid) < degree + 1:
        # 如果数据点不足，无法拟合多项式，使用线性插值作为备用
        group['value'] = group['value'].interpolate(method='linear', limit_direction='both')
    else:
        # 使用 numpy.polyfit 拟合多项式
        poly_coeffs = np.polyfit(valid['year'], valid['value'], degree)
        poly_func = np.poly1d(poly_coeffs)
        # 对整个组的 year 计算 value，包括缺失值
        group['value'] = poly_func(group['year'])
    # 限制值在 [0, 30] 范围内
    group['value'] = group['value'].clip(lower=0, upper=30)
    return group

# 应用多项式插值
df_interpolated = df_interpolated.groupby(['geogcode', 'variable']).apply(lambda g: polynomial_interpolate_group(g, degree=2)).reset_index(drop=True)

# 将插值结果保存到新表
df_interpolated.to_sql('SOC1020_sub_combined_interpolated', conn, if_exists='replace', index=False)

# 计算插值误差：对于有原始数据的点，计算拟合值与原始值的均方误差 (MSE)
error_df = df.copy()
error_df = error_df.merge(df_interpolated, on=['geogcode', 'year', 'variable'], suffixes=('_original', '_interpolated'))
error_df['error'] = (error_df['value_original'] - error_df['value_interpolated']) ** 2
error_df = error_df.dropna(subset=['value_original'])  # 只计算有原始数据的点

# 计算总体 MSE
mse = error_df['error'].mean()

# 计算每个 geogcode 的 MSE
mse_by_geogcode = error_df.groupby('geogcode')['error'].mean().reset_index()
mse_by_geogcode.columns = ['geogcode', 'mse']

# 打印总体 MSE
print("\n总体均方误差 (MSE):")
print(mse)

# 打印每个地区的 MSE
print("\n每个地区的均方误差 (MSE):")
print(mse_by_geogcode)

# 第二步：计算 2024 年每个地区的就业比例总和
df_2024_interpolated = df_interpolated[df_interpolated['year'] == 2024]
sum_by_region = df_2024_interpolated.groupby('geogcode')['value'].sum().reset_index()
sum_by_region.columns = ['geogcode', 'total_employment_proportion']

# 打印每个地区的总和
print("\n2024 年每个地区的就业比例总和：")
print(sum_by_region)

# 第三步：处理 2024 年仍为空值或为 0 的数据
# 找出 value 为空或为 0 的记录
df_2024_missing = df_2024_interpolated[df_2024_interpolated['value'].isna() | (df_2024_interpolated['value'] == 0)]
missing_counts = df_2024_missing.groupby('geogcode').size().reset_index(name='missing_count')

# 筛选只有单一空值或零值的地区
single_missing_regions = missing_counts[missing_counts['missing_count'] == 1]['geogcode']

# 处理单一空值或零值的地区
for region in single_missing_regions:
    # 获取该地区 2024 年的所有数据
    region_data = df_2024_interpolated[df_2024_interpolated['geogcode'] == region]
    # 计算非空且非零值的总和
    valid_values = region_data[region_data['value'].notna() & (region_data['value'] != 0)]['value']
    total_valid = valid_values.sum()
    # 如果总和小于 100，设置空值或零值为 100 - 总和
    if total_valid < 100:
        # 找到空值或零值的行
        missing_row = region_data[region_data['value'].isna() | (region_data['value'] == 0)]
        if not missing_row.empty:
            # 计算新值，限制在 [0, 30]
            new_value = min(max(100 - total_valid, 0), 30)
            # 更新 df_interpolated 中的值
            df_interpolated.loc[
                (df_interpolated['geogcode'] == region) &
                (df_interpolated['year'] == 2024) &
                (df_interpolated['variable'] == missing_row['variable'].iloc[0]),
                'value'
            ] = new_value

# 将更新后的插值表保存回数据库
df_interpolated.to_sql('SOC1020_sub_combined_interpolated', conn, if_exists='replace', index=False)

# 重新计算并打印最终的 2024 年各地区就业比例总和
df_2024_final = df_interpolated[df_interpolated['year'] == 2024]
final_sum_by_region = df_2024_final.groupby('geogcode')['value'].sum().reset_index()
final_sum_by_region.columns = ['geogcode', 'total_employment_proportion']

print("\n处理单一空值后的 2024 年各地区就业比例总和：")
print(final_sum_by_region)

# 检查仍为空值或为 0 的数据
remaining_issues = df_2024_final[df_2024_final['value'].isna() | (df_2024_final['value'] == 0)]
if not remaining_issues.empty:
    print("\n2024 年仍存在空值或零值的记录：")
    print(remaining_issues[['geogcode', 'year', 'variable', 'value']])
else:
    print("\n2024 年没有剩余的空值或零值。")

# 关闭数据库连接
conn.close()

(按照2024年Column Total比例进行补充)

In [ ]:
import sqlite3
import pandas as pd

# 连接到数据库
conn = sqlite3.connect('nomis_data.db')
cursor = conn.cursor()

# 步骤1：获取2024年Column Total的基准数据
query_column_total = """
SELECT variable, value
FROM SOC1020_sub_combined_interpolated
WHERE year = 2024 AND geogcode = 'Column Total'
"""
column_total_df = pd.read_sql_query(query_column_total, conn)

# 步骤2：获取2024年非Column Total的地区数据
query_region_data = """
SELECT geogcode, variable, COALESCE(value, 0) AS value
FROM SOC1020_sub_combined_interpolated
WHERE year = 2024 AND geogcode != 'Column Total'
"""
region_data_df = pd.read_sql_query(query_region_data, conn)

# 步骤3：补全缺失的variable
# 获取所有地区和所有可能的variable（从Column Total）
all_geogcodes = region_data_df['geogcode'].unique()
all_variables = column_total_df['variable'].unique()

# 创建所有geogcode和variable的组合
all_combinations = pd.MultiIndex.from_product(
    [all_geogcodes, all_variables], names=['geogcode', 'variable']
).to_frame(index=False)

# 合并region_data_df和all_combinations
region_data_df = all_combinations.merge(
    region_data_df, on=['geogcode', 'variable'], how='left'
)

# 步骤4：替换空值或0值为Column Total的基准值
# 合并Column Total的value
region_data_df = region_data_df.merge(
    column_total_df[['variable', 'value']], on='variable', how='left', suffixes=('', '_ref')
)
# 填充空值或0值
region_data_df['value'] = region_data_df.apply(
    lambda row: row['value_ref'] if pd.isna(row['value']) or row['value'] == 0 else row['value'],
    axis=1
)
# 删除临时的value_ref列
region_data_df = region_data_df.drop(columns=['value_ref'])

# 步骤5：归一化处理，确保每个地区value之和为100
# 计算每个地区的value总和
region_data_df['total_value'] = region_data_df.groupby('geogcode')['value'].transform('sum')
# 归一化value
region_data_df['normalized_value'] = region_data_df['value'] * 100.0 / region_data_df['total_value']
# 删除临时列
region_data_df = region_data_df.drop(columns=['value', 'total_value'])

# 步骤6：添加year列并重命名
region_data_df['year'] = 2024
region_data_df = region_data_df.rename(columns={'normalized_value': 'value'})

# 步骤7：保存结果到新表SOC1020_sub_combined_interpolated_2024
# 写入新表
#cursor.execute("DROP TABLE IF EXISTS SOC1020_sub_combined_2024")
region_data_df[['geogcode', 'year', 'variable', 'value']].to_sql(
    'SOC1020_sub_combined_interpolated_2024', conn, index=False, if_exists='replace'
)

# 提交更改并关闭连接
conn.commit()
conn.close()

print("处理完成，结果已保存到SOC1020_sub_combined_interpolated_2024表")

## 3.清理其他自变量数据

整理数据库nomis_data.db中的以下表格，输出表格都在原表名后面添加_cleaned:
1.commute_byoccupation_cleaned：按照Distance travelled to wor和geogcode分类，仅保留Occupation(current)中Total对应的value数据，输出Distance travelled to wor、geogcode、Date、value字段。
2.pay_time_weekly_cleaned:保留value type字段为number的部分，除了Confidence和value type字段删掉，其他字段都保留。
3.population_industry:保留value type字段为number的部分，除了Confidence、flag、Units、Population、value type字段删掉，其他字段都保留。
4.un_employment_qualification_byage：保留value type字段为percent的部分,除了Confidence、flag、value type字段删掉，其他字段都保留。

In [ ]:
import sqlite3
import pandas as pd

# 连接数据库
conn = sqlite3.connect('nomis_data.db')
cursor = conn.cursor()

# 1. 处理 commute_byoccupation
query_commute = """
SELECT "Distance travelled to wor", geogcode, Date, value
FROM commute_byoccupation
WHERE "Occupation (current)" = 'Total'
"""
commute_df = pd.read_sql_query(query_commute, conn)
cursor.execute("DROP TABLE IF EXISTS commute_byoccupation_cleaned")
commute_df.to_sql('commute_byoccupation_cleaned', conn, index=False, if_exists='replace')

# 2. 处理 pay_time_weekly
query_pay = """
SELECT *
FROM pay_time_weekly
WHERE "value type" = 'number'
"""
pay_df = pd.read_sql_query(query_pay, conn)
# 删除 Confidence 和 value type 字段
pay_df = pay_df.drop(columns=['Confidence','Sex', 'Item Name', 'value type', 'flag'], errors='ignore')
cursor.execute("DROP TABLE IF EXISTS pay_time_weekly_cleaned")
pay_df.to_sql('pay_time_weekly_cleaned', conn, index=False, if_exists='replace')

# 3. 处理 population_industry
query_pop = """
SELECT *
FROM population_industry
WHERE "value type" = '%'
AND "Industry (current)" != 'Total: All usual residents aged 16 years and over in employment the week before the census'
"""
pop_df = pd.read_sql_query(query_pop, conn)
# 删除 Confidence, flag, Units, Population, value type 字段
pop_df = pop_df.drop(columns=['Confidence', 'flag', 'Units', 'Population', 'value type'], errors='ignore')
cursor.execute("DROP TABLE IF EXISTS population_industry_cleaned")
pop_df.to_sql('population_industry_cleaned', conn, index=False, if_exists='replace')

# 4. 处理 un_employment_qualification_byage
query_unemp = """
SELECT *
FROM un_employment_qualification_byage
WHERE "value type" = 'Percent'
"""
unemp_df = pd.read_sql_query(query_unemp, conn)
# 删除 Confidence, flag, value type 字段
unemp_df = unemp_df.drop(columns=['Confidence', 'flag', 'value type'], errors='ignore')
cursor.execute("DROP TABLE IF EXISTS un_employment_qualification_byage_cleaned")
unemp_df.to_sql('un_employment_qualification_byage_cleaned', conn, index=False, if_exists='replace')

#5. 处理 population_byage
# 将population_byage表加载到DataFrame
popa_df = pd.read_sql_query("SELECT * FROM population_byage", conn)

# 透视数据，将“Total”和其他年龄组作为列
pivot_df = popa_df.pivot_table(
    index=['geogcode', 'Date'],
    columns='Age',
    values='value',
    aggfunc='sum'
).reset_index()
# 计算非“Total”年龄组的百分比
age_columns = [col for col in pivot_df.columns if col not in ['geogcode', 'Date', 'Total']]
for col in age_columns:
    pivot_df[col] = (pivot_df[col] / pivot_df['Total'] * 100).round(2)
# 将DataFrame转换回长格式
cleaned_popa_df = pivot_df.melt(
    id_vars=['geogcode', 'Date'],
    value_vars=age_columns,
    var_name='Age',
    value_name='value'
)
# 删除value为NaN的行
cleaned_popa_df = cleaned_popa_df.dropna(subset=['value'])
# 创建清理后的表
cursor.execute("DROP TABLE IF EXISTS population_byage_cleaned")
cleaned_popa_df.to_sql('population_byage_cleaned', conn, index=False)

#6. 处理 population_byage
# 将 population_byage 表加载到 DataFrame，仅选择指定字段
pop_dis_df = pd.read_sql_query(
    "SELECT geogcode, Date, Age, value FROM population_byage WHERE Age = 'Total'",conn)
# 将清洗后的 DataFrame 保存到 population_distirct_cleaned 表
pop_dis_df.to_sql('population_distirct_cleaned', conn, index=False, if_exists='replace')

# 提交更改并关闭连接
conn.commit()
conn.close()

print("所有表格处理完成，结果已保存到以下新表：")
print("- commute_byoccupation_cleaned")
print("- pay_time_weekly_cleaned")
print("- population_industry_cleaned")
print("- un_employment_qualification_byage_cleaned")
print("- population_byage_cleaned")
print("- population_distirct_cleaned")

In [ ]:
import sqlite3
import pandas as pd

# 连接数据库
conn = sqlite3.connect('nomis_data.db')
cursor = conn.cursor()

# 1. 处理 pay_time_weekly_cleaned
query_pay = """
SELECT *
FROM pay_time_weekly_cleaned
"""
pay_df = pd.read_sql_query(query_pay, conn)

# 1.1 拆分为 pay_weekly_cleaned (Weekly pay - gross)
pay_weekly_df = pay_df[pay_df['pay'] == 'Weekly pay - gross']
cursor.execute("DROP TABLE IF EXISTS pay_weekly_cleaned")
pay_weekly_df.to_sql('pay_weekly_cleaned', conn, index=False, if_exists='replace')

# 1.2 拆分为 hours_weekly_cleaned (Hours worked - total)
hours_weekly_df = pay_df[pay_df['pay'] == 'Hours worked - total']
cursor.execute("DROP TABLE IF EXISTS hours_weekly_cleaned")
hours_weekly_df.to_sql('hours_weekly_cleaned', conn, index=False, if_exists='replace')

# 2. 处理 un_employment_qualification_byage_cleaned
query_unemp = """
SELECT *
FROM un_employment_qualification_byage_cleaned
"""
unemp_df = pd.read_sql_query(query_unemp, conn)

# 2.1 拆分为 self_employment_cleaned (% aged 16-64 who are self employed)
self_emp_df = unemp_df[unemp_df['variable'] == '% aged 16-64 who are self employed']
cursor.execute("DROP TABLE IF EXISTS self_employment_cleaned")
self_emp_df.to_sql('self_employment_cleaned', conn, index=False, if_exists='replace')

# 2.3 拆分为 qualification_cleaned (RQF1+ to RQF4+)
qualification_vars = [
    '% with RQF4+ - aged 16-64',
    '% with RQF3+ - aged 16-64',
    '% with RQF2+ - aged 16-64',
    '% with RQF1+ - aged 16-64'
]
qual_df = unemp_df[unemp_df['variable'].isin(qualification_vars)]
cursor.execute("DROP TABLE IF EXISTS qualification_cleaned")
qual_df.to_sql('qualification_cleaned', conn, index=False, if_exists='replace')

# 提交更改并关闭连接
conn.commit()
conn.close()

print("表格拆分完成，结果已保存到以下新表：")
print("- pay_weekly_cleaned")
print("- hours_weekly_cleaned")
print("- self_employment_cleaned")
print("- qualification_cleaned")

# 二、指数构建

## 1.计算JPI

In [ ]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('nomis_data.db')

# Query to categorize and sum values by high, median, low
query = """
SELECT 
    geogcode,
    year,
    SUM(CASE WHEN variable IN ('1', '2', '3') THEN value ELSE 0 END) as high_sum,
    SUM(CASE WHEN variable IN ('8', '9') THEN value ELSE 0 END) as low_sum,
    SUM(CASE WHEN variable NOT IN ('1', '2', '3', '8', '9') THEN value ELSE 0 END) as median_sum
FROM SOC1020_combined_interpolated
GROUP BY geogcode, year
"""

# Load data into a DataFrame
df = pd.read_sql_query(query, conn)

# Calculate h_change and l_change
changes = df.groupby('geogcode').apply(
    lambda x: pd.Series({
        'h_change': x[x['year'] == 2024]['high_sum'].iloc[0] - x[(x['year'] >= 2019) & (x['year'] <= 2023)]['high_sum'].mean(),
        'l_change': x[x['year'] == 2024]['low_sum'].iloc[0] - x[(x['year'] >= 2019) & (x['year'] <= 2023)]['low_sum'].mean()
    })
).reset_index()

# Calculate JPI
changes['JPI'] = (changes['h_change'] + changes['l_change']) * (1 + abs(changes['h_change'] - changes['l_change'])) * 0.5 * 0.01

# Select relevant columns
result = changes[['geogcode', 'h_change', 'l_change', 'JPI']].sort_values('geogcode')

# Create new table employment_JPI
result.to_sql('employment_JPI', conn, if_exists='replace', index=False)

# Close the connection
conn.close()

## 2.计算AI暴露度

In [ ]:
import sqlite3
import pandas as pd

# 步骤1：定义AI暴露指数表
ai_exposure_data = {
    'SOC2010_sub_map': [
        '11', '12', '21', '22', '23', '24', '31', '32', '33', '34', '35', '41', '42',
        '51', '52', '53', '54', '61', '62', '71', '81', '82', '91', '92'
    ],
    'Average_AI_Exposure': [
        0.6302, 0.3593, 0.9020, 0.4754, 0.7698, 0.8288, 0.9478, 0.5080, 0.2940, 0.5338,
        0.7494, 0.5630, 0.8065, 0.5196, 0.4529, 0.1760, 0.3752, 0.2880, 0.2017, 0.0880,
        0.3674, 0.2720, 0.1260, 0.0293
    ]
}
ai_exposure_df = pd.DataFrame(ai_exposure_data)

# 步骤2：连接数据库并读取2024年数据
conn = sqlite3.connect('nomis_data.db')
query = """
SELECT geogcode, variable, value
FROM SOC1020_sub_combined_interpolated_2024
WHERE year = 2024
"""
data_df = pd.read_sql_query(query, conn)

# 步骤3：合并AI暴露指数
merged_df = data_df.merge(ai_exposure_df, left_on='variable', right_on='SOC2010_sub_map', how='left')

# 步骤4：计算每个geogcode的加权平均AI暴露指数
# 计算value * AI_exposure
merged_df['weighted_exposure'] = merged_df['value'] * merged_df['Average_AI_Exposure']
# 按geogcode分组，计算加权平均
result_df = merged_df.groupby('geogcode').agg({
    'weighted_exposure': 'sum',
    'value': 'sum'
}).reset_index()
# 计算平均AI暴露指数（value已归一化为100，无需除以sum(value)）
result_df['avg_ai_exposure'] = result_df['weighted_exposure'] / 100.0
# 选择所需列
result_df = result_df[['geogcode', 'avg_ai_exposure']]

# 步骤5：保存结果到新表
cursor = conn.cursor()
#cursor.execute("DROP TABLE IF EXISTS interpolation_report")
result_df.to_sql('AI_avg_exposure_2024', conn, index=False, if_exists='replace')

# 提交更改并关闭连接
conn.commit()
conn.close()

print("处理完成，结果已保存到AI_avg_exposure_2024表")

## 3.2024年计算高低技能职业就业率

In [ ]:
import sqlite3
import pandas as pd

# 连接到SQLite数据库
conn = sqlite3.connect('nomis_data.db')

# 创建新表occupation_high_low的SQL语句
create_table_sql = """
CREATE TABLE occupation_high_low AS
SELECT 
    geogcode AS geogcode,
    SUM(CASE WHEN variable IN (1, 2, 3) THEN value ELSE 0 END) AS high_occupation,
    SUM(CASE WHEN variable IN (8, 9) THEN value ELSE 0 END) AS low_occupation
FROM 
    (SELECT * FROM SOC1020_combined_interpolated WHERE year = 2024)
GROUP BY 
    geogcode;
"""

# 执行SQL语句
try:
    cursor = conn.cursor()
    cursor.execute(create_table_sql)
    conn.commit()
    print("表 occupation_high_low 创建成功！")

finally:
    conn.close()

# 三、空间自相关检验（Moran’s I）与建模OLS\SAR\SEM\SDM

## <u>统计检验与比较</u>
(1) 空间依赖性诊断
Moran's I检验：检验OLS残差是否存在空间自相关。若显著，说明需空间模型。

拉格朗日乘数检验（LM test）：

LM-lag：检验SAR是否优于OLS。

LM-error：检验空间误差模型（SEM）是否优于OLS。

稳健性检验（Robust LM）：当LM-lag和LM-error均显著时使用。

似然比检验（LR test）：比较SDM与SAR或SEM，验证SDM是否需要包含WX。

需获取：上述检验的统计量和p值。

(2) 模型拟合优度
对数似然值（Log-Likelihood）：值越大，模型拟合越好。

AIC/BIC：越低越好，平衡拟合优度和复杂度（SDM通常参数更多，需惩罚过拟合）。

R²：但空间模型的R²解释力有限，需结合其他指标。

需比较：各模型的AIC/BIC和似然值。



## 1.准备gdf_ols数据

In [1]:
import sqlite3
import pandas as pd

# 连接到数据库
conn = sqlite3.connect('nomis_data.db')
cursor = conn.cursor()

# 列出所有需要处理的表格
tables = [
    'commute_byoccupation_cleaned',
    'population_industry_cleaned',
    'pay_weekly_cleaned',
    'hours_weekly_cleaned',
    'self_employment_cleaned',
    'qualification_cleaned',
    'population_byage_cleaned'
]

# 加载Al_avg_exposure_2024表
base_df1 = pd.read_sql_query("SELECT * FROM AI_avg_exposure_2024", conn)
#base_df1.to_csv('AI_avg_exposure_2024.csv', index=False, encoding='utf-8-sig')
base_df2 = pd.read_sql_query("SELECT * FROM employment_JPI", conn)
#base_df2.to_csv('employment_JPI.csv', index=False, encoding='utf-8-sig')
#population_distirct_cleaned
base_df3 = pd.read_sql_query("SELECT * FROM population_distirct_cleaned", conn)
base_df3 = base_df3.rename(columns={'value': 'population_total'})
base_df4 = pd.read_sql_query("SELECT * FROM unemployment_rate_cleaned", conn)
# 初始化结果DataFrame，仅包含geogcode
result_df = base_df1

# 对每个表格进行处理
for table in tables:
    # 加载表格
    df = pd.read_sql_query(f"SELECT * FROM {table}", conn)
    
    # 获取分类列（排除geogcode和date）
    category_cols = [col for col in df.columns if col not in ['geogcode', 'Date', 'value']]
    
    if category_cols:
        # 假设只有一个分类列
        category_col = category_cols[0]
        # 透视表格
        pivot_df = df.pivot_table(
            index='geogcode',
            columns=category_col,
            values='value',
            aggfunc='first'
        ).reset_index()
        
        # 重命名列以反映来源表
        pivot_df.columns = ['geogcode'] + [f"{col}" for col in pivot_df.columns[1:]]
        
        # 与结果DataFrame合并
        result_df = result_df.merge(pivot_df, on='geogcode', how='left')
    else:
        # 如果没有分类列，直接使用value
        pivot_df = df[['geogcode', 'value']].groupby('geogcode').first().reset_index()
        pivot_df.columns = ['geogcode', f"{table}_value"]
        result_df = result_df.merge(pivot_df, on='geogcode', how='left')

# 合并AI_avg_exposure_2024的数据
result_df = result_df.rename(columns={'value': 'AI_avg_exposure_2024'})
result_df = result_df.merge(base_df2[['geogcode','JPI']], on='geogcode')
result_df = result_df.merge(base_df3[['geogcode','population_total']], on='geogcode')
result_df = result_df.merge(base_df4[['geogcode','value']], on='geogcode')
result_df = result_df.rename(columns={'geogcode': 'LAD25CD','value':'unemployment_rate'})

# 创建新表
cursor.execute("DROP TABLE IF EXISTS ols_data_2024")
result_df.to_sql('ols_data_2024', conn, index=False)

# 提交并关闭连接
conn.commit()
conn.close()

In [2]:
import geopandas as gpd
import numpy as np
import sqlite3
import pandas as pd
try:    
    conn = sqlite3.connect('nomis_data.db')
    cursor = conn.cursor()  
    # 读取数据
    # query = """
    # SELECT *
    # FROM ols_data_2024
    # WHERE LAD25CD NOT IN ('E06000053', 'E09000001')
    # """
    query = """
    SELECT *
    FROM ols_data_2024
    """
    df = pd.read_sql_query(query, conn)
    df['LAD25CD'] = df['LAD25CD'].astype(str)

    gdf_shp = gpd.read_file("D:/Dissertation/project/data/spatial data/LAD_MAY_2025_UK_BFC_2360005762104150824/LAD_MAY_2025_UK_BFC.shp")
    gdf = gdf_shp.merge(
        df, 
        on="LAD25CD",  # 确保两边列名一致
        how="left"     # 保留所有行政区，即使统计数据缺失
    )


except sqlite3.Error as e:
    print(f"读取 ols_data_2024 表失败: {e}")
    conn.close()
    exit()

# 删除空值
gdf_ols = gdf.replace({'% aged 16-64 who are self employed': '!'}, np.nan)
# 定义需要跳过的列（几何列 + 其他指定列）
skip_cols = ['LAD25CD', 'LAD25NM', 'LAD25NMW', 'BNG_E', 'BNG_N', 'LONG', 'LAT','GlobalID', 'geometry']
# 仅对非跳过列进行 float 转换
for col in gdf_ols.columns:
    if col not in skip_cols:  # 跳过几何列和指定列
        gdf_ols[col] = pd.to_numeric(gdf_ols[col], errors='coerce')


## 2.样本分组

In [3]:
from sklearn.preprocessing import RobustScaler
#分组建模数据集
#1.就业指标
occ=gdf_ols[['LAD25CD', 'LAD25NM', 'LAD25NMW', 'BNG_E', 'BNG_N', 'LONG', 'LAT','GlobalID', 'geometry',
    'avg_ai_exposure', 'JPI','10km to less than 30km', '30km and over','Less than 10km', 'Works mainly from home',
    'Weekly pay - gross', 'Hours worked - total'
    #'unemployment_rate', '% aged 16-64 who are self employed', 
    ]]
#清除空缺值
check_null_cols = [col for col in occ.columns if col not in skip_cols]
occ = occ.dropna(subset=check_null_cols)
#稳健标准化
columns_to_scale = ['10km to less than 30km','30km and over','Less than 10km', 'Works mainly from home','Weekly pay - gross', 'Hours worked - total',
    #'unemployment_rate' 
    ]
robust_scaler = RobustScaler()
occ[columns_to_scale] = robust_scaler.fit_transform(occ[columns_to_scale])
#自变量值
occ_X=['JPI','10km to less than 30km', '30km and over','Less than 10km', 'Works mainly from home',
    'Weekly pay - gross', 'Hours worked - total',
    #'unemployment_rate', '% aged 16-64 who are self employed'
    ]

#2.产业指标
ind=gdf_ols[['LAD25CD', 'LAD25NM', 'LAD25NMW', 'BNG_E', 'BNG_N', 'LONG', 'LAT','GlobalID', 'geometry',
    'avg_ai_exposure',
    'A: Agriculture, Forestry and fishing', 'B: Mining and quarrying',
    'C: Manufacturing',
    'D: Electricity, gas, steam and air conditioning supply',
    'E:  Water supply; Sewerage, Waste management and Remediation activities',
    'F: Construction',
    'G: Wholesale and retail trade; repair of motor vehicles and motorcycles',
    'H: Transport and storage',
    'I: Accommodation and food service activities',
    'J: Information and communication',
    'K: Financial and insurance activities', 'L: Real estate activities',
    'M: Professional, scientific and technical activities',
    'N: Administrative and support service activities',
    'O: Public administration and defence; compulsory social security',
    'P: Education', 'Q: Human health and social work activities',
    'R, S, T, U Other']]
# 中心化行业就业比例（Industry Employment Share Centralization）
industry_cols = [
    'A: Agriculture, Forestry and fishing', 'B: Mining and quarrying',
    'C: Manufacturing',
    'D: Electricity, gas, steam and air conditioning supply',
    'E:  Water supply; Sewerage, Waste management and Remediation activities',
    'F: Construction',
    'G: Wholesale and retail trade; repair of motor vehicles and motorcycles',
    'H: Transport and storage',
    'I: Accommodation and food service activities',
    'J: Information and communication',
    'K: Financial and insurance activities', 'L: Real estate activities',
    'M: Professional, scientific and technical activities',
    'N: Administrative and support service activities',
    'O: Public administration and defence; compulsory social security',
    'P: Education', 'Q: Human health and social work activities',
    'R, S, T, U Other',
    ]
national_means = ind[industry_cols].mean()
# 创建新列存放“中心化后”的行业就业比例
for col in industry_cols:
    ind[f'{col}'] = ind[col] - national_means[col]
#清除空值
check_null_cols = [col for col in ind.columns if col not in skip_cols]
ind = ind.dropna(subset=check_null_cols)
#建模自变量
ind_X=['A: Agriculture, Forestry and fishing', 'B: Mining and quarrying',
    'C: Manufacturing',
    'D: Electricity, gas, steam and air conditioning supply',
    'E:  Water supply; Sewerage, Waste management and Remediation activities',
    'F: Construction',
    'G: Wholesale and retail trade; repair of motor vehicles and motorcycles',
    'H: Transport and storage',
    'I: Accommodation and food service activities',
    'J: Information and communication',
    'K: Financial and insurance activities', 'L: Real estate activities',
    'M: Professional, scientific and technical activities',
    'N: Administrative and support service activities',
    'O: Public administration and defence; compulsory social security',
    'P: Education', 'Q: Human health and social work activities','R, S, T, U Other'
    ]

#3.人口指标
pop=gdf_ols[['LAD25CD', 'LAD25NM', 'LAD25NMW', 'BNG_E', 'BNG_N', 'LONG', 'LAT','GlobalID', 'geometry',
    'avg_ai_exposure',
    '% with RQF1+ - aged 16-64',
    '% with RQF2+ - aged 16-64', '% with RQF3+ - aged 16-64',
    '% with RQF4+ - aged 16-64',
    'Aged 25 to 34 years', 'Aged 35 to 49 years',
    'Aged 50 to 64 years', 'Aged 65 years and over',
    'population_total']]
#均一化
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# 对population_total单独处理（如需要乘以100）
pop['population_total'] = scaler.fit_transform(pop[['population_total']]) * 100
check_null_cols = [col for col in pop.columns if col not in skip_cols]
pop = pop.dropna(subset=check_null_cols)
#定义自变量
pop_X=['% with RQF1+ - aged 16-64','% with RQF2+ - aged 16-64', 
    '% with RQF3+ - aged 16-64', '% with RQF4+ - aged 16-64',
    'Aged 25 to 34 years', 'Aged 35 to 49 years',
    'Aged 50 to 64 years', 'Aged 65 years and over',
    'population_total']

f:\Python 3.13\Lib\site-packages\geopandas\geodataframe.py:1968: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
f:\Python 3.13\Lib\site-packages\geopandas\geodataframe.py:1968: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


## 3.构建组合

In [4]:
import pandas as pd
import statsmodels.api as sm
from itertools import combinations
from tqdm import tqdm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from spreg import OLS
#1.ols模拟

def ols_with_spatial_tests(X_df, y, w, max_vars):
    """
    带空间检验的OLS全组合回归
    
    参数:
        X_df (pd.DataFrame): 原始自变量（不含常数项）
        y (pd.Series): 因变量
        w (libpysal.weights.W): 空间权重矩阵
        max_vars (int): 最大变量组合数
    返回:
        pd.DataFrame: 包含空间检验的结果
    """
    features = X_df.columns.tolist()
    results = []
    
    for k in range(1, min(max_vars, len(features)) + 1):
        for combo in tqdm(combinations(features, k), desc=f'{k}变量组合'):
            X_subset = sm.add_constant(X_df[list(combo)])
            
            # 使用spreg的OLS获取空间检验
            model = OLS(
                y.values.reshape(-1, 1),
                X_subset.values,
                w=w,
                name_y='avg_ai_exposure',
                name_x=['const'] + list(combo),
                spat_diag=True
            )            
            # 提取结果
            result = {
                '变量数': k,
                '变量组合': ', '.join(combo),
                'R²': model.r2,
                '调整R²': model.ar2,
                'AIC': model.aic,
                'BIC': model.schwarz,
                'LM_ERR': model.lm_error[0],
                'LM_ERR_p': model.lm_error[1],
                'LM_LAG': model.lm_lag[0],
                'LM_LAG_p': model.lm_lag[1],
                'Robust_LM_ERR': model.rlm_error[0],
                'Robust_LM_ERR_p': model.rlm_error[1],
                'Robust_LM_LAG': model.rlm_lag[0],
                'Robust_LM_LAG_p': model.rlm_lag[1],
                'SARMA': model.lm_sarma[0],
                'SARMA_p': model.lm_sarma[1],
                'Log likelihood': model.logll
                }              
            
            # 添加变量系数和P值（使用t_stat替代z_stat）
            for i, var in enumerate(combo):
                result[f'coef_{var}'] = model.betas[i+1][0]  # 跳过常数项
                result[f'p_{var}'] = model.t_stat[i+1][1]    # 使用t_stat获取p值
            
            # 新增字段：数p值<0.05的变量个数
            significant_count = sum(1 for key in result if key.startswith('p_') and result[key] < 0.05)
            result['p值<0.05变量个数'] = significant_count
            
            results.append(result)
    
    return pd.DataFrame(results)


In [5]:
#2.SEM模拟
from spreg import ML_Error

def sem_with_spatial_tests(X_df, y, w, max_vars):
    """
    带空间检验的SEM全组合回归
    
    参数:
        X_df (pd.DataFrame): 原始自变量（不含常数项）
        y (pd.Series): 因变量
        w (libpysal.weights.W): 空间权重矩阵
        max_vars (int): 最大变量组合数
    返回:
        pd.DataFrame: 包含空间检验的结果
    """
    features = X_df.columns.tolist()
    results = []
    
    for k in range(1, min(max_vars, len(features)) + 1):
        for combo in tqdm(combinations(features, k), desc=f'{k}变量组合'):
            X_subset = sm.add_constant(X_df[list(combo)])
            
            # 使用spreg的OLS获取空间检验
            model = ML_Error(
                y.values.reshape(-1, 1),
                X_subset.values,
                w=w,
                name_y='avg_ai_exposure',
                name_x=['const'] + list(combo),
                spat_diag=True
            )            
            # 提取结果
            result = {
                '变量数': k,
                '变量组合': ', '.join(combo),
                'R²': model.pr2,
                'AIC': model.aic,
                'BIC': model.schwarz,
                'Sigma²': model.sig2,
                '空间系数(lambda)': model.lam,
                'lambda_z值': model.z_stat[-1][0],  # 最后一个是lambda的统计量
                'lambda_p值': model.z_stat[-1][1],
                '均值_y': model.mean_y,
                '标准差_y': model.std_y
                }      
                        
            # 添加变量系数和统计量
            significant_count = 0
            for i, var in enumerate(['const'] + list(combo)):
                if i < len(model.betas)-1:  # 排除lambda的系数
                    result[f'coef_{var}'] = model.betas[i][0]
                    result[f'std_err_{var}'] = model.std_err[i]
                    result[f'z_{var}'] = model.z_stat[i][0]
                    result[f'p_{var}'] = model.z_stat[i][1]
                    
                    if var != 'const' and result[f'p_{var}'] < 0.05:
                        significant_count += 1
            
            # 添加诊断信息
            result.update({
                'p值<0.05变量个数': significant_count,
                '显著变量占比': f"{significant_count/k:.1%}",
                '残差平方和': model.utu
            })
                        
            results.append(result)
    
    return pd.DataFrame(results)


In [6]:
#3.SAR模拟
from spreg import ML_Lag

def sar_with_spatial_tests(X_df, y, w, max_vars):
    """
    带空间检验的SAR全组合回归
    
    参数:
        X_df (pd.DataFrame): 原始自变量（不含常数项）
        y (pd.Series): 因变量
        w (libpysal.weights.W): 空间权重矩阵
        max_vars (int): 最大变量组合数
    返回:
        pd.DataFrame: 包含空间检验的结果
    """
    features = X_df.columns.tolist()
    results = []
    
    for k in range(1, min(max_vars, len(features)) + 1):
        for combo in tqdm(combinations(features, k), desc=f'{k}变量组合'):
            X_subset = sm.add_constant(X_df[list(combo)])
            
            # 使用spreg的OLS获取空间检验
            model = ML_Lag(
                y.values.reshape(-1, 1),
                X_subset.values,
                w=w,
                name_y='avg_ai_exposure',
                name_x=['const'] + list(combo),
                spat_diag=True
            )            
            # 提取结果
            result = {
                '变量数': k,
                '变量组合': ', '.join(combo),
                'R²': model.pr2,                
                'R²_e': model.pr2_e,
                'AIC': model.aic,
                'BIC': model.schwarz,
                'Sigma²': model.sig2,
                '均值_y': model.mean_y,
                '标准差_y': model.std_y
                }
            
            # 添加变量系数和统计量
            significant_count = 0
            for i, var in enumerate(['const'] + list(combo)):
                if i < len(model.betas)-1:  # 排除lambda的系数
                    result[f'coef_{var}'] = model.betas[i][0]
                    result[f'std_err_{var}'] = model.std_err[i]
                    result[f'z_{var}'] = model.z_stat[i][0]
                    result[f'p_{var}'] = model.z_stat[i][1]
                    
                    if var != 'const' and result[f'p_{var}'] < 0.05:
                        significant_count += 1
            
            # 添加诊断信息
            result.update({
                'p值<0.05变量个数': significant_count,
                '显著变量占比': f"{significant_count/k:.1%}",
                '残差平方和': model.utu
            })
                        
            results.append(result)
    
    return pd.DataFrame(results)


In [7]:
#4.SDM模拟
from spreg import GM_Combo  # 改为使用空间杜宾模型

def sdm_with_spatial_tests(X_df, y, w, max_vars, w_lags=1, slx_lags=1, vcm=True):
    """
    空间杜宾模型(SDM)全组合回归
    
    Parameters:
        X_df (pd.DataFrame): 原始自变量（不含常数项）
        y (pd.Series): 因变量
        w (libpysal.weights.W): 空间权重矩阵
        max_vars (int): 最大变量组合数
        w_lags (int): W的滞后阶数作为工具变量（默认1）
        slx_lags (int): X的空间滞后阶数（默认1）
        vcm (bool): 是否包含方差协方差矩阵（默认False）
    
    Returns:
        pd.DataFrame: 包含SDM模型结果的空间检验
    """
    features = X_df.columns.tolist()
    results = []
    
    for k in range(1, min(max_vars, len(features)) + 1):
        for combo in tqdm(combinations(features, k), desc=f'{k}变量组合'):
            try:
                X_subset = sm.add_constant(X_df[list(combo)])
                
                # 构建SDM模型（通过GNSM实现）
                model = GM_Combo(
                    y.values.reshape(-1, 1),
                    X_subset.values,
                    w=w,
                    w_lags=w_lags,
                    slx_lags=slx_lags,
                    name_y=y.name if hasattr(y, 'name') else 'y',
                    name_x=['const'] + list(combo),
                    vm=vcm
                )
                
                # 提取结果，使用getattr安全访问
                result = {
                    '变量数': k,
                    '变量组合': ', '.join(combo),
                    'R²': getattr(model, 'pr2', None),  # 使用pr2作为备用
                    'pr2_e': getattr(model, 'pr2_e', None),  # 安全提取pr2_e
                    # 'Log likelihood': getattr(model, 'logll', None)  # 注释掉或设为None，因为无logll
                }
                
                # 添加变量系数和P值（GM_Combo使用z_stat而非t_stat）
                for i, var in enumerate(combo):
                    result[f'coef_{var}'] = model.betas[i+1][0] if hasattr(model, 'betas') else None
                    result[f'p_{var}'] = model.z_stat[i+1][1] if hasattr(model, 'z_stat') else None  # GM_Combo用z_stat
                
                # 新增字段：数p值<0.05的变量个数
                significant_count = sum(1 for key in result if key.startswith('p_') and result[key] is not None and result[key] < 0.05)
                result['p值<0.05变量个数'] = significant_count
                
                results.append(result)
                
            except Exception as e:
                print(f"组合 {combo} 建模失败: {str(e)}")
                continue
    
    # 转换为DataFrame并处理排序
    df_results = pd.DataFrame(results)
    
    return df_results

## (1)就业指标

### 变量组合尝试

In [8]:
import libpysal
from libpysal.weights import Queen
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from spreg import OLS

# 1.构建几何邻接关系
queen_w = Queen.from_dataframe(occ)  # 默认基于几何边界
queen_w.transform = 'r'  # 行标准化（推荐）
# 检查孤岛（无邻居的区域）
isolated = np.where(np.array([len(queen_w.neighbors[i]) for i in range(queen_w.n)]) == 0)[0]
print(f"孤岛数量：{len(isolated)}，索引：{isolated}")
# 剔除孤岛（可选）
if len(isolated) > 0:
    mask = np.ones(queen_w.n, dtype=bool)
    mask[isolated] = False
    occ = occ.iloc[mask]
    queen_w_occ = libpysal.weights.Queen.from_dataframe(occ)
else:
    queen_w_occ = queen_w  

queen_w_occ.transform = 'r'

C:\Users\uiol\AppData\Local\Temp\ipykernel_25232\285459991.py:10: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  queen_w = Queen.from_dataframe(occ)  # 默认基于几何边界
f:\Python 3.13\Lib\site-packages\libpysal\weights\contiguity.py:347: UserWarning: The weights matrix is not fully connected: 
 There are 3 disconnected components.
 There are 2 islands with ids: 43, 293.
  W.__init__(self, neighbors, ids=ids, **kw)
C:\Users\uiol\AppData\Local\Temp\ipykernel_25232\285459991.py:20: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  queen_w_occ = libpysal.weights.Queen.from_dataframe(occ)


('WARNING: ', 43, ' is an island (no neighbors)')
('WARNING: ', 293, ' is an island (no neighbors)')
孤岛数量：2，索引：[ 43 293]


### 几何邻接关系Queen方法进行最小二乘法OLS建模

In [9]:
# 执行分析
occ_ols_test = ols_with_spatial_tests(
    X_df=occ[occ_X].astype(float),
    y=occ['avg_ai_exposure'],
    w=queen_w_occ,  # 确保已定义空间权重矩阵
    max_vars=7
)
# 保存完整结果
#occ_ols_test.to_excel('ols_with_spatial_tests.xlsx', index=False)

1变量组合: 7it [00:00, 72.00it/s]
2变量组合: 21it [00:00, 95.75it/s]
3变量组合: 35it [00:00, 90.05it/s]
4变量组合: 35it [00:00, 89.65it/s]
5变量组合: 21it [00:00, 75.87it/s]
6变量组合: 7it [00:00, 73.53it/s]
7变量组合: 1it [00:00, 63.71it/s]


In [10]:
#2.构建ols模型
# 添加常数项（保持为 DataFrame）
from sklearn.discriminant_analysis import StandardScaler
occ_X_modify=['JPI','10km to less than 30km', '30km and over','Less than 10km', 'Works mainly from home','Weekly pay - gross']
X_df_modify = occ[occ_X_modify].astype(float)
X = sm.add_constant(X_df_modify)
# 因变量
Y = np.array(occ['avg_ai_exposure'], dtype=float).reshape(-1, 1)

# # Ridge 回归（自动交叉验证调参）
# # 标准化
# scaler = StandardScaler()
# X_scaled_R = scaler.fit_transform(X)
# # 转换为 NumPy 数组用于 OLS（仅在必要时）
# X = np.array(X, dtype=float)
# 生成自变量名称列表
name_x = ['const'] + occ_X_modify
# 运行 OLS 回归
ols = OLS(
    name_ds="occ",
    y=Y,
    x=X,
    w=queen_w_occ,
    name_y="avg_ai_exposure",  # 明确因变量名
    name_x=name_x,    # 使用筛选后的自变量名
    name_w=queen_w_occ,
    moran=True,
    spat_diag=True
)
# 打印 OLS 结果
print("============")
print("\nOLS 回归摘要")
print(ols.summary)


OLS 回归摘要
REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :         occ
Weights matrix      :<libpysal.weights.contiguity.Queen object at 0x000002249DA3A210>
Dependent Variable  :avg_ai_exposure                Number of Observations:         313
Mean dependent var  :      0.5056                Number of Variables   :           7
S.D. dependent var  :      0.0529                Degrees of Freedom    :         306
R-squared           :      0.5873
Adjusted R-squared  :      0.5792
Sum squared residual:    0.360609                F-statistic           :     72.5829
Sigma-square        :       0.001                Prob(F-statistic)     :   6.414e-56
S.E. of regression  :       0.034                Log likelihood        :     614.777
Sigma-square ML     :       0.001                Akaike info criterion :   -1215.553
S.E of regression ML:      0.0339                Schwarz criterion     :   -1189.

#### OLS 回归摘要概述

这份报告展示了普通最小二乘法 (OLS) 回归分析的结果，使用的数据集名为“occ”。因变量是“avg_ai_exposure”（可能代表平均 AI 暴露度或类似指标），其均值为 0.5056，标准差为 0.0529，观测样本数为 313。该模型包含 7 个变量（包括常数项），自由度为 306。报告中提到了基于 Queen 邻接的权重矩阵，暗示了空间分析元素，但主要模型是非空间的 OLS。

关键模型拟合指标：
- **R-squared**：0.5873（表明独立变量解释了因变量变异的约 58.73%）。
- **Adjusted R-squared**：0.5792（调整后值稍低，用于惩罚过拟合）。
- **F-statistic**：72.5829，p 值 6.414e-56（高度显著，拒绝所有系数为零的原假设）。
- **残差平方和**：0.360609。
- **Sigma-square (MSE)**：0.001（均方误差）。
- **回归标准误差**：0.034。
- **对数似然值**：614.777。
- **AIC**：-1215.553（Akaike 信息准则，用于模型比较；值越低越好）。
- **BIC**：-1189.330（Schwarz 准则，与 AIC 类似，但对复杂性惩罚更重）。

警告：变量 ['const'] 因为是常量而被移除，但模型仍包含 CONSTANT 项。

系数估计与显著性

下表总结了每个变量的回归系数、标准误差、t 统计量和 p 值。所有变量在 5% 水平（p < 0.05）上统计显著，其中大多数在更严格水平（p < 0.001）上显著。正系数表示与 avg_ai_exposure 的正相关，反之负相关。

| 变量                  | 系数       | 标准误差   | t 统计量   | p 值     | 解释 |
|-----------------------|------------|------------|------------|----------|------|
| CONSTANT             | 0.49825   | 0.00230   | 216.34857 | 0.00000 | 当所有其他变量为零时的基线 avg_ai_exposure 水平。高度显著。 |
| JPI                  | 0.01232   | 0.00359   | 3.43690   | 0.00067 | 正效应；JPI（可能为工作接近指数）每单位增加会使暴露度提高 0.01232。在 0.1% 水平显著。 |
| 10km to less than 30km | -0.00556 | 0.00280   | -1.98549  | 0.04798 | 相对于基线的负效应；此距离类别使暴露度降低 0.00556。在 5% 水平边缘显著。 |
| 30km and over        | -0.00501 | 0.00207   | -2.42613  | 0.01584 | 更强的负效应；更长距离使暴露度降低 0.00501。在 5% 水平显著。 |
| Less than 10km       | -0.02803 | 0.00353   | -7.93511  | 0.00000 | 最大的负效应；短距离使暴露度降低 0.02803。高度显著。 |
| Works mainly from home | 0.04712 | 0.00387   | 12.17167  | 0.00000 | 强的正效应；主要在家工作使暴露度增加 0.04712。高度显著。 |
| Weekly pay - gross   | 0.01431   | 0.00292   | 4.90391   | 0.00000 | 正效应；每周毛工资每单位增加使暴露度提高 0.01431。高度显著。 |

这些变量似乎捕捉了诸如工作接近度 (JPI)、通勤距离（类别虚拟变量，可能相对于未显示的参考类别，如无通勤或城市核心）、远程工作状态和收入（每周毛工资）等因素。距离变量表明，较近或在家工作模式对 AI 暴露度的影响不同，可能与职业或经济背景相关。

 回归诊断

 多重共线性
- **条件数**：4.757（低值；通常 <30 表示预测变量间无严重多重共线性问题）。

 残差正态性
- **Jarque-Bera 测试**：值 0.129，自由度 2，p=0.9375（无法拒绝正态性原假设；残差似乎呈正态分布）。

 异方差性（非恒定方差）
- **Breusch-Pagan 测试**：值 24.285，自由度 6，p=0.0005（拒绝同方差原假设；存在异方差证据）。
- **Koenker-Bassett 测试**：值 25.556，自由度 6，p=0.0003（稳健版本；也表明异方差，暗示方差随预测变量变化）。

这意味着标准误差可能有偏差；可以使用稳健标准误差或加权最小二乘法来改善推断。

 空间依赖诊断
鉴于空间权重矩阵，测试检查残差或滞后的空间自相关：

- **SARERR（空间自回归误差模型）测试**：
  - Moran's I (残差)：0.0857，值 2.525，p=0.0116（残差中存在显著空间自相关）。
  - LM (滞后)：6.166，自由度 1，p=0.0130（暗示空间滞后模型可能合适）。
  - 稳健 LM (滞后)：1.335，自由度 1，p=0.2479（调整后不显著）。
  - LM (误差)：5.082，自由度 1，p=0.0242（暗示空间误差模型）。
  - 稳健 LM (误差)：0.251，自由度 1，p=0.6167（不显著）。
  - LM (SARMA)：6.417，自由度 2，p=0.0404（组合滞后/误差模型显著）。

- **空间 Durbin 模型 (SDM) 测试**（包括 X 变量的空间滞后）：
  - LM for WX：5.833，自由度 6，p=0.4422（无证据支持预测变量的空间滞后）。
  - 稳健 LM WX：4.749，自由度 6，p=0.5764（类似不显著）。
  - LM (滞后)：6.166，自由度 1，p=0.0130（与之前一致）。
  - 稳健 LM Lag - SDM：5.082，自由度 1，p=0.0242。
  - SDM 联合测试：10.915，自由度 7，p=0.1424（整体对 SDM 不显著）。

总体而言，存在空间依赖证据（特别是在误差或滞后中），表明空间模型（如空间滞后或误差模型）可能比纯 OLS 更适合，以考虑数据中的地理聚类。

 含义与建议
该模型解释了相当大的变异部分，但存在异方差性和空间依赖问题，如果未处理可能导致推断偏差。变量突出了远程工作和接近度如何与 AI 暴露度正相关，而较长通勤则负相关。为了稳健性，考虑空间计量模型或异方差稳健估计量。如果这是更大分析的一部分（如按地区划分的职业 AI 暴露），进一步诊断或变量转换可能必要。

### 多重共线性与异方差检验：VIF、Breusch–Pagan test

In [11]:
from statsmodels.stats.diagnostic import het_breuschpagan

# 计算 VIF 报告
# 确保 X_selected_array 有列名，转换为 DataFrame 以匹配
X_selected_df = pd.DataFrame(X, columns=name_x)
vif_data = pd.DataFrame()
vif_data["Variable"] = X_selected_df.columns

vif_data["VIF"] = [variance_inflation_factor(X_selected_df.values, i) 
for i in range(X_selected_df.shape[1])]
print("VIF 报告：")
print(vif_data.sort_values(by="VIF", ascending=False).to_string(index=False))

# 1. 首先获取模型的残差
residuals = ols.u  # pysal OLS模型的残差存储在u属性中
print(f"残差获取完成，长度：{len(residuals)}")
# 2. 准备解释变量矩阵（与原始模型相同）
exog = X
print(f"解释变量矩阵准备完成，形状：{exog.shape}")
# 3. 执行Breusch-Pagan检验
bp_test = het_breuschpagan(residuals, exog)
# 4. 输出结果
labels = ['LM Statistic', 'LM-Test p-value', 'F-Statistic', 'F-Test p-value']
bp_results = dict(zip(labels, bp_test))
print("Breusch-Pagan 检验结果：")
for label, value in bp_results.items():
    print(f"{label}: {value:.4f}")

VIF 报告：
              Variable      VIF
Works mainly from home 3.816596
10km to less than 30km 3.612301
        Less than 10km 3.148183
         30km and over 2.519375
    Weekly pay - gross 1.717233
                 const 1.408662
                   JPI 1.026107
残差获取完成，长度：313
解释变量矩阵准备完成，形状：(313, 7)
Breusch-Pagan 检验结果：
LM Statistic: 22.1602
LM-Test p-value: 0.0011
F-Statistic: 3.8859
F-Test p-value: 0.0009


在传统的回归模型中，空间依赖性未被建模时，残差中隐含的空间模式会表现为“异方差”（即使真实误差方差恒定）。很明显在全局莫兰指数和局部莫兰指数中可以发现，因变量AI暴露指数有这很强的空间聚类分布特征。

### SEM空间误差模型

In [12]:
# 执行分析
occ_sem_test = sem_with_spatial_tests(
    X_df=occ[occ_X].astype(float),
    y=occ['avg_ai_exposure'],
    w=queen_w_occ,  # 确保已定义空间权重矩阵
    max_vars=7
)
# 保存完整结果
#occ_sem_test.to_excel('ols_with_spatial_tests.xlsx', index=False)

1变量组合: 0it [00:00, ?it/s]f:\Python 3.13\Lib\site-packages\spreg\ml_error.py:184: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  res = minimize_scalar(
1变量组合: 1it [00:00,  1.49it/s]f:\Python 3.13\Lib\site-packages\spreg\ml_error.py:184: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  res = minimize_scalar(
1变量组合: 2it [00:00,  2.95it/s]f:\Python 3.13\Lib\site-packages\spreg\ml_error.py:184: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  res = minimize_scalar(
1变量组合: 3it [00:00,  3.92it/s]f:\Python 3.13\Lib\site-packages\spreg\ml_error.py:184: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  res = minimize_scalar(
1变量组合: 4it [00:01,  5.03it/s]f:\Python 3.13\Lib\site-packages\spreg\ml_error.py:184: RuntimeWarning: Method 'bounded' does not s

In [13]:
from spreg import ML_Error
# 3. 运行 sem 模型
sem = ML_Error(
    name_ds="occ",
    y=Y,
    x=X,
    w=queen_w_occ,
    name_y="avg_ai_exposure",
    name_x=name_x,
    name_w=queen_w_occ,
    spat_diag=True,
    slx_lags=1,  # 生成 WX
    robust='white'
)
print("=== SEM 模型结果 ===")
print(sem.summary)

print("\n=== 模型比较  ===")
print(f"OLS AIC : {ols.aic}")
print(f"SEM AIC : {sem.aic}")

f:\Python 3.13\Lib\site-packages\spreg\ml_error.py:184: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  res = minimize_scalar(


=== SEM 模型结果 ===
REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ML SPATIAL ERROR WITH SLX (SLX-Error) (METHOD = full)
------------------------------------------------------------------------
Data set            :         occ
Weights matrix      :<libpysal.weights.contiguity.Queen object at 0x000002249DA3A210>
Dependent Variable  :avg_ai_exposure                Number of Observations:         313
Mean dependent var  :      0.5056                Number of Variables   :          13
S.D. dependent var  :      0.0529                Degrees of Freedom    :         300
Pseudo R-squared    :      0.5946
Log likelihood      :    620.4065
Sigma-square ML     :      0.0011                Akaike info criterion :   -1214.813
S.E of regression   :      0.0332                Schwarz criterion     :   -1166.112

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------

#### 模型概述
这是一个最大似然（Maximum Likelihood）空间误差模型（Spatial Error Model, SEM）的回归结果，使用的数据集名为"occ"，包含313个观测样本，与之前的OLS模型相同。空间权重矩阵采用Queen邻接方式，表明模型考虑了残差的空间自相关（通过lambda参数捕捉）。因变量仍是"avg_ai_exposure"（平均AI暴露度），均值0.5056，标准差0.0529。模型包括7个自变量（加上常量项）和lambda（空间误差系数），自由度为306。

模型的整体拟合度：
- **Pseudo R-squared**：0.5872，类似于OLS的R-squared（0.5873），表示解释了约58.72%的变异，但Pseudo R-squared在空间模型中更注重似然而非残差平方和。
- **对数似然（Log likelihood）**：617.0733，高于OLS的614.777，表示SEM拟合更好。
- **AIC（Akaike info criterion）**：-1220.147，比OLS的-1215.553更低（更优），用于模型选择。
- **BIC（Schwarz criterion）**：-1193.923，比OLS的-1189.330更低。
- **Sigma-square ML**：0.0011，回归标准误（S.E. of regression）：0.0336，略低于OLS的0.034，表示残差波动更小。

报告中有一个警告："Variable(s) ['const'] removed for being constant." 与OLS相同，可能在处理过程中常量项被临时移除但在最终估计中保留。末尾的RuntimeWarning是Python代码执行的提示（spreg库中的优化器警告），不影响结果解释，但建议检查数值稳定性。

SEM模型方程大致为：

avg_ai_exposure = 0.49780 + 0.01256 * JPI - 0.00523 * (10km to less than 30km) - 0.00496 * (30km and over) - 0.02858 * (Less than 10km) + 0.04739 * (Works mainly from home) + 0.01352 * (Weekly pay - gross) + u

其中u = lambda * W u + ε（W为空间权重矩阵，lambda为空间自相关系数，ε为独立残差）。

系数解释
系数表列出了每个自变量的估计值、标准误、z统计量（因ML估计，使用z而非t）和p值。大多数系数在5%显著性水平下显著，但"10km to less than 30km"的p=0.06264略高于0.05（在10%水平显著）。空间参数lambda=0.16874，z=2.13754，p=0.03255，表示残差存在正空间自相关（相邻观测的误差相关），证实了OLS诊断中的空间依赖问题。

与OLS相比，SEM的系数略有调整（如JPI从0.01232增至0.01256，Weekly pay从0.01431降至0.01352），但方向和显著性类似，表明空间修正后估计更稳健。

| 变量                  | 系数      | 标准误 | z统计量 | p值     | 解释                                                                 |
|-----------------------|-----------|--------|---------|---------|----------------------------------------------------------------------|
| CONSTANT             | 0.49780  | 0.00261| 190.98 | 0.00000 | 截距项：基准AI暴露度约为0.498，与OLS类似。                          |
| JPI                  | 0.01256  | 0.00352| 3.57   | 0.00036 | 正向影响：JPI每增加1单位，AI暴露度增加0.01256，略高于OLS。         |
| 10km to less than 30km | -0.00523| 0.00281| -1.86  | 0.06264 | 负向影响：10-30km距离的个体AI暴露度降低0.00523，显著性边际（p>0.05，但<0.1）。 |
| 30km and over        | -0.00496 | 0.00217| -2.29  | 0.02206 | 负向影响：30km以上距离的个体AI暴露度降低0.00496，与OLS类似。       |
| Less than 10km       | -0.02858 | 0.00353| -8.09  | 0.00000 | 强负向影响：小于10km距离的个体AI暴露度降低0.02858，幅度略大于OLS。 |
| Works mainly from home | 0.04739| 0.00393| 12.05  | 0.00000 | 正向影响：主要在家工作者AI暴露度增加0.04739，略高于OLS。           |
| Weekly pay - gross   | 0.01352  | 0.00302| 4.48   | 0.00001 | 正向影响：周薪每增加1单位，AI暴露度增加0.01352，略低于OLS。         |
| lambda               | 0.16874  | 0.07894| 2.14   | 0.03255 | 空间误差自相关：正值表示相邻区域残差正相关，模型成功捕捉空间依赖。 |

这些系数表明，距离变量（可能为通勤距离虚拟变量）的负效应、在家工作的正效应和高薪的正效应保持一致。lambda的显著性验证了空间误差的存在，忽略它（如在OLS中）可能导致偏差。

与OLS模型的比较
- **拟合度提升**：SEM的Log likelihood更高（617.073 vs. 614.777），AIC更低（-1220.147 vs. -1215.553），表明SEM在考虑空间依赖后拟合更好。根据AIC差异（约4.59），SEM优于OLS（差异>2通常表示显著改进）。
- **系数变化**：大多数系数幅度略微调整，但符号一致。"10km to less than 30km"在SEM中显著性减弱（从p=0.048到0.063），可能因空间修正吸收了部分变异。标准误有些变化（如Weekly pay的标准误从0.00292增至0.00302），反映ML估计的差异。
- **空间依赖修正**：OLS诊断显示残差Moran's I显著（p=0.0116）和LM(error)显著（p=0.0242），SEM通过lambda直接处理此问题，提高估计效率。
- **其他**：Pseudo R-squared几乎相同，表明空间效应主要影响残差而非解释力。SEM的S.E. of regression更小（0.0336 vs. 0.034），残差更精确。

总体评估与建议
- **优势**：SEM有效修正了OLS中的空间依赖和异方差问题（隐含通过ML估计），系数可靠，所有关键变量显著。模型适合空间数据，如区域AI暴露度分析。
- **问题**：一个距离变量的显著性边际，可能需进一步检验。报告无额外诊断（如多重共线性或残差正态），假设从OLS转移适用，但建议验证。RuntimeWarning提示优化方法可能不精确，考虑其他求解器（如'scipy'而非'bounded'）。
- **改进**：1) 与空间滞后模型（SLM）比较，看是否滞后因变量更合适；2) 使用似然比检验正式比较OLS vs. SEM；3) 如果数据允许，加入交互项或更多空间诊断；4) 解释变量含义（如JPI是什么指数，距离基准组是什么）以增强实际洞见。

总体而言，SEM是OLS的改进版，更适合存在空间自相关的场景，提供更准确的AI暴露度影响估计。

### SAR空间滞后模型

In [14]:
# 执行分析
occ_sar_test = sar_with_spatial_tests(
    X_df=occ[occ_X].astype(float),
    y=occ['avg_ai_exposure'],
    w=queen_w_occ,  # 确保已定义空间权重矩阵
    max_vars=7
)
# 保存完整结果
#occ_sem_test.to_excel('ols_with_spatial_tests.xlsx', index=False)

1变量组合: 7it [00:01,  6.43it/s]
2变量组合: 21it [00:03,  6.43it/s]
3变量组合: 35it [00:04,  7.51it/s]
4变量组合: 35it [00:03,  9.23it/s]
5变量组合: 21it [00:03,  6.99it/s]
6变量组合: 7it [00:00, 10.79it/s]
7变量组合: 1it [00:00,  8.60it/s]


In [15]:
from spreg import ML_Lag
# 4. 运行 slm 模型
slm = ML_Lag(
    name_ds="occ",
    y=Y,
    x=X,
    w=queen_w_occ,
    name_y="avg_ai_exposure",
    name_x=name_x,
    name_w=queen_w_occ,    
    spat_diag=True,
    slx_lags=1,  # 生成 WX
    robust='white'
)
print("=== SAR 模型结果 ===")
print(slm.summary)

print("\n=== 模型比较  ===")
print(f"OLS AIC : {ols.aic}")
print(f"SAR AIC : {slm.aic}")

=== SAR 模型结果 ===
REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG WITH SLX - SPATIAL DURBIN MODEL (METHOD = FULL)
-------------------------------------------------------------------------------------------------
Data set            :         occ
Weights matrix      :<libpysal.weights.contiguity.Queen object at 0x000002249DA3A210>
Dependent Variable  :avg_ai_exposure                Number of Observations:         313
Mean dependent var  :      0.5056                Number of Variables   :          14
S.D. dependent var  :      0.0529                Degrees of Freedom    :         299
Pseudo R-squared    :      0.6035
Spatial Pseudo R-squared:  0.5943
Log likelihood      :    620.1047
Sigma-square ML     :      0.0011                Akaike info criterion :   -1212.209
S.E of regression   :      0.0333                Schwarz criterion     :   -1159.762

------------------------------------------------------------------------------------
            

#### 模型概述
这是一个最大似然空间滞后模型（Maximum Likelihood Spatial Lag Model, SAR，也称为Spatial Autoregressive Model）的回归结果，使用的数据集名为"occ"，包含313个观测样本，与之前的OLS和SEM模型相同。空间权重矩阵采用Queen邻接方式，模型通过加入空间滞后因变量（W_avg_ai_exposure）来捕捉因变量的空间依赖（即相邻区域的AI暴露度影响本地暴露度）。因变量仍是"avg_ai_exposure"（平均AI暴露度），均值0.5056，标准差0.0529。模型包括8个变量（7个自变量加上常量项，以及空间滞后项），自由度为305。

模型的整体拟合度：
- **Pseudo R-squared**：0.5965，表示解释了约59.65%的变异，高于OLS的0.5873和SEM的0.5872。
- **Spatial Pseudo R-squared**：0.5889，专注于空间调整后的解释力。
- **对数似然（Log likelihood）**：617.5991，高于SEM的617.0733和OLS的614.777，表示SAR拟合更好。
- **AIC（Akaike info criterion）**：-1219.198，比OLS的-1215.553更低（更优），但略高于SEM的-1220.147。
- **BIC（Schwarz criterion）**：-1189.229，与SEM的-1193.923相比稍高，但优于OLS的-1189.330。
- **Sigma-square ML**：0.0011，回归标准误（S.E. of regression）：0.0336，与SEM相同，低于OLS的0.034，表示残差波动更小。

报告中有一个警告："Variable(s) ['const'] removed for being constant." 与前模型相同，可能在处理过程中常量项被临时移除但在最终估计中保留。

SAR模型方程大致为：

avg_ai_exposure = 0.42382 + 0.01184 * JPI - 0.00536 * (10km to less than 30km) - 0.00438 * (30km and over) - 0.02645 * (Less than 10km) + 0.04420 * (Works mainly from home) + 0.01126 * (Weekly pay - gross) + 0.14746 * W_avg_ai_exposure + ε

其中W_avg_ai_exposure是空间权重矩阵W与因变量的乘积（空间滞后项），系数ρ=0.14746（在表中为W_avg_ai_exposure），z=2.33788，p=0.01939，表示正空间自相关：相邻区域的AI暴露度每增加1单位，本地暴露度增加0.14746。

 系数解释
系数表列出了每个自变量的估计值、标准误、z统计量（因ML估计，使用z而非t）和p值。所有系数在5%显著性水平下显著（p<0.05），空间滞后项也显著。相比OLS和SEM，SAR的截距项显著降低（从约0.498降至0.424），其他系数幅度缩小（如Weekly pay从OLS的0.01431降至0.01126），因为部分效应被空间滞后吸收。

| 变量                  | 系数      | 标准误 | z统计量 | p值     | 解释                                                                 |
|-----------------------|-----------|--------|---------|---------|----------------------------------------------------------------------|
| CONSTANT             | 0.42382  | 0.03173| 13.36  | 0.00000 | 截距项：基准AI暴露度约为0.424，低于前模型，反映空间调整。           |
| JPI                  | 0.01184  | 0.00351| 3.37   | 0.00074 | 正向影响：JPI每增加1单位，AI暴露度增加0.01184，略低于前模型。       |
| 10km to less than 30km | -0.00536| 0.00274| -1.96  | 0.05019 | 负向影响：10-30km距离的个体AI暴露度降低0.00536，显著性边际（p≈0.05）。 |
| 30km and over        | -0.00438 | 0.00204| -2.14  | 0.03205 | 负向影响：30km以上距离的个体AI暴露度降低0.00438，与前模型类似。     |
| Less than 10km       | -0.02645 | 0.00357| -7.42  | 0.00000 | 强负向影响：小于10km距离的个体AI暴露度降低0.02645，幅度小于前模型。 |
| Works mainly from home | 0.04420| 0.00407| 10.85  | 0.00000 | 正向影响：主要在家工作者AI暴露度增加0.04420，低于前模型。           |
| Weekly pay - gross   | 0.01126  | 0.00306| 3.68   | 0.00023 | 正向影响：周薪每增加1单位，AI暴露度增加0.01126，显著低于前模型。     |
| W_avg_ai_exposure    | 0.14746  | 0.06307| 2.34   | 0.01939 | 空间滞后：相邻区域AI暴露度的正溢出效应，模型成功捕捉空间依赖。     |

这些系数表明，距离变量的负效应、在家工作的正效应和高薪的正效应保持一致，但空间滞后项解释了部分变异，导致自变量系数缩小。

 空间滞后模型影响（Impacts）
SAR模型独特地提供了影响分解，使用'simple'方法（基于系数和ρ的简单计算，可能不如模拟方法精确，但快速）。影响分为：
- **直接影响（Direct）**：自变量对本地因变量的即时效应（类似于系数，但略微调整）。
- **间接影响（Indirect）**：自变量通过空间滞后对相邻区域的溢出效应。
- **总影响（Total）**：直接+间接。

| 变量                  | 直接影响 | 间接影响 | 总影响  | 解释                                                                 |
|-----------------------|-----------|----------|---------|----------------------------------------------------------------------|
| JPI                  | 0.0118   | 0.0020  | 0.0139 | JPI的总效应为0.0139，其中约85%为直接，15%为空间溢出。              |
| 10km to less than 30km | -0.0054 | -0.0009 | -0.0063| 负总效应-0.0063，间接效应放大直接负影响。                           |
| 30km and over        | -0.0044  | -0.0008 | -0.0051| 负总效应-0.0051，类似模式。                                         |
| Less than 10km       | -0.0264  | -0.0046 | -0.0310| 强负总效应-0.0310，间接效应贡献约15%。                             |
| Works mainly from home | 0.0442  | 0.0076  | 0.0518 | 正总效应0.0518，间接效应显著（约15%），表示在家工作有区域溢出。    |
| Weekly pay - gross   | 0.0113   | 0.0019  | 0.0132 | 正总效应0.0132，间接效应放大直接影响。                               |

这些影响突显空间依赖的重要性：忽略间接效应（如在OLS中）会低估总影响（例如，在家工作的总效应比系数高17%）。

与OLS和SEM模型的比较
- **拟合度提升**：SAR的Log likelihood最高（617.599 vs. SEM 617.073 vs. OLS 614.777），Pseudo R-squared最高（0.5965 vs. SEM 0.5872 vs. OLS 0.5873）。AIC：SEM最低（-1220.147，最优），SAR次之（-1219.198），OLS最差（-1215.553）。AIC差异（SAR vs. OLS ≈3.64，表明SAR优于OLS；SAR vs. SEM ≈0.95，SEM略优）。S.E. of regression相同于SEM，优于OLS。
- **系数变化**：SAR系数普遍小于OLS和SEM（例如，Weekly pay：SAR 0.01126 < SEM 0.01352 < OLS 0.01431），因为空间滞后吸收了部分正向变异。截距显著降低，反映空间调整。"10km to less than 30km"在SAR中p=0.05019（边际显著），类似于SEM的0.06264，但优于OLS的0.04798。空间参数：SAR的ρ=0.14746（滞后因变量，p=0.019） vs. SEM的lambda=0.16874（滞后残差，p=0.033），两者均捕捉正空间依赖，但SAR焦点在因变量依赖。
- **空间依赖修正**：OLS诊断显示LM(lag)显著（p=0.013），支持SAR。SAR通过滞后因变量直接处理此问题，而SEM处理残差依赖。SAR的Spatial Pseudo R-squared表明空间贡献约0.5889的解释力。
- **影响与解释**：SAR独有影响分解，提供更丰富的洞见（如间接效应），而SEM和OLS仅给平均效应。SAR适合因变量空间依赖强的场景（例如，AI暴露度的区域扩散）。
- **其他**：所有模型残差波动类似，SAR的自由度略低（305 vs. 306），因额外参数。警告一致。

| 模型 | AIC       | Log Likelihood | Pseudo/R-squared | 空间参数显著性 | 关键优势                     |
|------|-----------|----------------|------------------|----------------|------------------------------|
| OLS | -1215.553| 614.777       | 0.5873          | 无             | 简单，但忽略空间依赖         |
| SEM | -1220.147| 617.073       | 0.5872          | lambda=0.169 (p=0.033)| 最佳AIC，修正残差空间依赖   |
| SAR | -1219.198| 617.599       | 0.5965          | ρ=0.147 (p=0.019)     | 最高似然，提供影响分解       |

 总体评估与建议
- **优势**：SAR有效修正了OLS中的空间滞后依赖（从诊断LM(lag)显著），拟合优于OLS，提供直接/间接影响以理解空间溢出（如高薪或在家工作的区域效应）。适合政策分析，例如评估通勤距离对AI暴露度的网络影响。
- **问题**：AIC略高于SEM，表明SEM可能在惩罚额外参数后更 parsimonious。"10km to less than 30km"显著性边际，需谨慎解释。影响使用'simple'方法，可能需'full'或模拟方法验证。
- **改进**：1) 使用似然比检验比较SAR vs. SEM（例如，共同嵌套模型如SDM）；2) 检查空间权重敏感性（例如，尝试Rook替代Queen）；3) 如果间接效应关键，优先SAR；否则SEM更简洁；4) 加入变量交互或诊断残差自相关以确认模型充分性。

总体而言，SAR是OLS的显著改进，与SEM竞争，提供互补洞见：SEM在整体拟合略胜，SAR在解释空间机制更强。根据AIC，SEM为最佳，但SAR的更高似然和影响分解使其在空间动态分析中更有价值。

### SDM空间杜宾模型

In [16]:
# 执行分析
occ_sdm_test = sdm_with_spatial_tests(
    X_df=occ[occ_X].astype(float),
    y=occ['avg_ai_exposure'],
    w=queen_w_occ,  # 确保已定义空间权重矩阵
    max_vars=7
)
# 保存完整结果
#occ_sem_test.to_excel('ols_with_spatial_tests.xlsx', index=False)

1变量组合: 7it [00:00, 43.07it/s]
2变量组合: 21it [00:00, 54.28it/s]
3变量组合: 35it [00:00, 55.76it/s]
4变量组合: 35it [00:00, 59.19it/s]
5变量组合: 21it [00:00, 52.92it/s]
6变量组合: 7it [00:00, 50.03it/s]
7变量组合: 1it [00:00, 37.61it/s]


In [17]:
from spreg import GM_Combo

# 拟合SDM模型
sdm = GM_Combo(
    name_ds="occ",
    y=Y,
    x=X,
    w=queen_w_occ,
    name_y="avg_ai_exposure",
    name_x=name_x,
    name_w=queen_w_occ,
    slx_lags=1,  # 生成 WX
)
print("=== SDM 模型结果 ===")
print(sdm.summary)


=== SDM 模型结果 ===
REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: SPATIALLY WEIGHTED 2SLS - GM-COMBO MODEL WITH SLX (GNSM)
---------------------------------------------------------------------------
Data set            :         occ
Weights matrix      :<libpysal.weights.contiguity.Queen object at 0x000002249DA3A210>
Dependent Variable  :avg_ai_exposure                Number of Observations:         313
Mean dependent var  :      0.5056                Number of Variables   :          14
S.D. dependent var  :      0.0529                Degrees of Freedom    :         299
Pseudo R-squared    :      0.3648
Spatial Pseudo R-squared: omitted due to rho outside the boundary (-1, 1).

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT         1.11506         0.43

#### 模型概述
这是一个空间Durbin模型（Spatial Durbin Model, SDM）的回归结果，使用空间加权二阶段最小二乘（Spatially Weighted 2SLS）方法，具体为GM-Combo模型结合SLX（Spatial Lag of X，自变量的空间滞后）。这是一个广义嵌套空间模型（GNSM），包括SLX项（W_ prefixed变量）、空间滞后因变量（W_avg_ai_exposure，rho）和空间误差（lambda）。数据集名为"occ"，包含313个观测样本，与之前模型相同。空间权重矩阵采用Queen邻接方式。因变量仍是"avg_ai_exposure"（平均AI暴露度），均值0.5056，标准差0.0529。模型包括14个变量（7个自变量、6个SLX项、常量项、rho和lambda），自由度为299。

模型的整体拟合度：
- **Pseudo R-squared**：0.3648，表示解释了约36.48%的变异，显著低于OLS的0.5873、SEM的0.5872和SAR的0.5965，表明SDM在纳入更多空间项后拟合较差（可能过拟合或空间项不合适）。
- **Spatial Pseudo R-squared**：省略，因为rho（W_avg_ai_exposure = -1.24681）超出(-1,1)边界，这是一个严重警告：通常rho应在(-1,1)内以确保模型稳定性和收敛性；超出可能表示模型设定错误、多重共线性或数据问题，导致解释力计算无效。
- 报告无Log likelihood、AIC或BIC，可能因为2SLS方法而非ML估计（不像SEM/SAR）。Sigma-square ML和S.E. of regression也未提供。

报告中警告：
- "Variable(s) ['const'] removed for being constant." 与前模型相同。
- "*** WARNING: Estimate for spatial lag coefficient is outside the boundary (-1, 1). ***"：rho=-1.24681超出边界，模型可能不稳定，建议检查空间权重、变量选择或使用其他估计方法。

SDM模型方程大致为：

avg_ai_exposure = 1.11506 + 0.00858 * JPI - 0.00693 * (10km to less than 30km) - 0.00470 * (30km and over) - 0.02501 * (Less than 10km) + 0.04455 * (Works mainly from home) + 0.01237 * (Weekly pay - gross) + 0.00445 * W_JPI - 0.00767 * W_(10km to less than 30km) - 0.00863 * W_(30km and over) - 0.02747 * W_(Less than 10km) + 0.05627 * W_(Works mainly from home) + 0.02226 * W_(Weekly pay - gross) - 1.24681 * W_avg_ai_exposure + u

其中u = lambda * W u + ε（lambda=0.85784为空间误差自相关，无标准误/z/p值，可能固定或从GM估计得出）。W_avg_ai_exposure被工具化，使用二阶滞后（W2_）作为仪器变量，以处理内生性（空间滞后因变量可能与残差相关）。

 系数解释
系数表列出了每个变量的估计值、标准误、z统计量（2SLS使用z）和p值。核心自变量大多显著，但所有SLX项（W_ prefixed）不显著（p>0.05），表明自变量的空间滞后效应弱。rho=-1.24681（p=0.15638，不显著且超出边界），lambda=0.85784（无显著性统计）。相比前模型，自变量系数变化（如JPI从SAR的0.01184降至0.00858），截距显著增加（从约0.42-0.50升至1.115），可能因SLX和负rho吸收变异。

| 变量                  | 系数      | 标准误 | z统计量 | p值     | 解释                                                                 |
|-----------------------|-----------|--------|---------|---------|----------------------------------------------------------------------|
| CONSTANT             | 1.11506  | 0.43881| 2.54   | 0.01105 | 截距项：基准AI暴露度约为1.115，远高于前模型，反映复杂空间调整可能导致偏差。 |
| JPI                  | 0.00858  | 0.00418| 2.05   | 0.04037 | 正向影响：JPI每增加1单位，AI暴露度增加0.00858，低于前模型，边际显著。 |
| 10km to less than 30km | -0.00693| 0.00290| -2.39  | 0.01689 | 负向影响：10-30km距离的个体AI暴露度降低0.00693，与前类似但幅度更大。 |
| 30km and over        | -0.00470 | 0.00246| -1.91  | 0.05619 | 负向影响：30km以上距离的个体AI暴露度降低0.00470，显著性边际（p>0.05，但<0.1）。 |
| Less than 10km       | -0.02501 | 0.00380| -6.59  | 0.00000 | 强负向影响：小于10km距离的个体AI暴露度降低0.02501，幅度小于前模型。 |
| Works mainly from home | 0.04455| 0.00412| 10.80  | 0.00000 | 正向影响：主要在家工作者AI暴露度增加0.04455，与SAR类似。           |
| Weekly pay - gross   | 0.01237  | 0.00408| 3.03   | 0.00243 | 正向影响：周薪每增加1单位，AI暴露度增加0.01237，介于SEM和SAR之间。   |
| W_JPI                | 0.00445  | 0.02617| 0.17   | 0.86501 | SLX：相邻JPI的效应不显著，正向但弱。                                |
| W_10km to less than 30km | -0.00767| 0.00711| -1.08  | 0.28052 | SLX：相邻10-30km距离的负溢出，不显著。                              |
| W_30km and over      | -0.00863 | 0.00469| -1.84  | 0.06586 | SLX：相邻30km以上距离的负溢出，边际显著（p<0.1）。                  |
| W_Less than 10km     | -0.02747 | 0.02580| -1.06  | 0.28689 | SLX：相邻小于10km距离的负溢出，不显著。                             |
| W_Works mainly from home | 0.05627| 0.03141| 1.79   | 0.07327 | SLX：相邻在家工作的正溢出，边际显著（p<0.1）。                      |
| W_Weekly pay - gross | 0.02226  | 0.02105| 1.06   | 0.29040 | SLX：相邻周薪的正溢出，不显著。                                     |
| W_avg_ai_exposure    | -1.24681 | 0.87967| -1.42  | 0.15638 | rho：空间滞后因变量，负值且不显著，超出(-1,1)边界，表示潜在负空间依赖但模型问题。 |
| lambda               | 0.85784  | -      | -       | -       | 空间误差自相关：正值，表示残差正空间依赖，但无显著性统计。         |

这些系数表明，核心自变量效应与前模型一致（距离负、在家工作和高薪正），但SLX项大多无贡献，rho负且无效，可能SDM过参数化。

 与OLS、SEM和SAR模型的比较
- **拟合度**：SDM的Pseudo R-squared最低（0.3648 vs. SAR 0.5965 > SEM 0.5872 ≈ OLS 0.5873），表明加入SLX和组合空间项未改善拟合，反而可能恶化（过拟合）。无AIC可比，但基于Pseudo R-squared，SDM最差。前模型有AIC（SEM最佳-1220.147，SAR-1219.198，OLS-1215.553），SDM缺乏但似然未报告，暗示不优。
- **系数变化**：核心自变量系数缩小（如JPI: SDM 0.00858 < SAR 0.01184 < SEM 0.01256 < OLS 0.01232），截距暴增（1.115 vs. 前约0.42-0.50），可能因负rho和SLX吸收。"30km and over"在SDM中p=0.056（边际），类似于SEM的0.022和SAR的0.032。"10km to less than 30km"保持显著但p略高（0.017 vs. 前0.048-0.063）。SLX项引入新维度，但大多不显著，类似于SAR诊断中LM WX不显著（p=0.442），不支持强SLX。
- **空间依赖修正**：SDM嵌套SAR（rho+SLX）和SEM（lambda+SLX），但rho超出边界且不显著，lambda高（0.858），表明强残差依赖但模型不稳定。前OLS诊断支持滞后（LM lag p=0.013）和误差（LM error p=0.024），SDM尝试组合但失败（警告）。工具变量使用二阶滞后处理内生性，这是2SLS的优势，但rho无效削弱。
- **空间参数**：SDM的rho=-1.247 (p=0.156，不显著) vs. SAR的ρ=0.147 (p=0.019，正显著)；lambda=0.858 vs. SEM的0.169 (p=0.033)。负rho不寻常，可能表示负溢出但模型问题。SLX项弱，与SAR Joint test for SDM (p=0.142，不支持SDM)一致。
- **其他**：SDM自由度最低（299 vs. 前305-306），参数多（14 vs. 前7-8），可能导致效率损失。无残差诊断，但警告暗示需验证稳定性。SDM未提供影响分解（如SAR），限制解释。

| 模型 | Pseudo/R-squared | 空间参数 | AIC (若有) | 关键优势/问题                  |
|------|------------------|----------|------------|--------------------------------|
| OLS | 0.5873          | 无       | -1215.553 | 简单，但忽略空间依赖           |
| SEM | 0.5872          | lambda=0.169 (p=0.033) | -1220.147 | 最佳AIC，修正残差依赖         |
| SAR | 0.5965          | ρ=0.147 (p=0.019) | -1219.198 | 最高R-squared，提供影响分解    |
| SDM | 0.3648          | rho=-1.247 (p=0.156, 超出边界); lambda=0.858 | 无        | 嵌套模型，但拟合差、参数不稳  |

 总体评估与建议
- **优势**：SDM理论上最全面（嵌套SAR和SEM，加上SLX），捕捉直接、间接和总效应（虽未报告）。核心效应一致，工具变量处理内生性。
- **问题**：拟合最差，rho超出边界（不稳定，可能无效模型），SLX项大多不显著（无额外价值），"30km and over"边际显著。警告表明潜在规格错误，如空间权重不当或多重共线性。
- **改进**：1) 检查rho边界问题，可能标准化权重或用ML代替2SLS；2) 似然比/ Wald检验比较嵌套模型（e.g., SDM vs. SAR/SEM）；3) 如果SLX不显著，退回SAR或SEM（基于前诊断，SAR/SEM更好）；4) 验证仪器变量有效性（e.g., Sargan检验）；5) 考虑数据问题，如异常值或小样本偏误。SDM不推荐为最终模型；SEM或SAR更可靠，提供更好拟合和解释。

总体而言，SDM尝试扩展空间结构但失败，导致拟合下降和不稳定性。前模型（尤其是SEM）优于SDM，用于AI暴露度分析更合适。

## (2)人口指标

### 变量组合尝试

In [18]:
#1.pop_ols
# 1.构建几何邻接关系
queen_w = libpysal.weights.Queen.from_dataframe(pop)  # 默认基于几何边界
queen_w.transform = 'r'  # 行标准化（推荐）
# 检查孤岛（无邻居的区域）
isolated = np.where(np.array([len(queen_w.neighbors[i]) for i in range(queen_w.n)]) == 0)[0]
print(f"孤岛数量：{len(isolated)}，索引：{isolated}")
# 剔除孤岛（可选）
if len(isolated) > 0:
    mask = np.ones(queen_w.n, dtype=bool)
    mask[isolated] = False
    pop = pop.iloc[mask]
    queen_w_pop = libpysal.weights.Queen.from_dataframe(pop)
else:
    queen_w_pop = queen_w    

queen_w_pop.transform = 'r'

C:\Users\uiol\AppData\Local\Temp\ipykernel_25232\1835648829.py:3: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  queen_w = libpysal.weights.Queen.from_dataframe(pop)  # 默认基于几何边界
f:\Python 3.13\Lib\site-packages\libpysal\weights\contiguity.py:347: UserWarning: The weights matrix is not fully connected: 
 There are 3 disconnected components.
 There are 2 islands with ids: 43, 292.
  W.__init__(self, neighbors, ids=ids, **kw)
C:\Users\uiol\AppData\Local\Temp\ipykernel_25232\1835648829.py:13: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  queen_w_pop = libpysal.weights.Queen.from_dataframe(pop)


('WARNING: ', 43, ' is an island (no neighbors)')
('WARNING: ', 292, ' is an island (no neighbors)')
孤岛数量：2，索引：[ 43 292]


### Queen&OLS建模

In [19]:
# 执行分析
pop_ols_test = ols_with_spatial_tests(
    X_df=pop[pop_X].astype(float),
    y=pop['avg_ai_exposure'],
    w=queen_w_pop,  # 确保已定义空间权重矩阵
    max_vars=9
)

1变量组合: 9it [00:00, 92.61it/s]
2变量组合: 36it [00:00, 86.29it/s]
3变量组合: 84it [00:00, 93.73it/s]
4变量组合: 126it [00:01, 92.05it/s]
5变量组合: 126it [00:01, 89.72it/s]
6变量组合: 84it [00:00, 89.39it/s]
7变量组合: 36it [00:00, 84.98it/s]
8变量组合: 9it [00:00, 86.61it/s]
9变量组合: 1it [00:00, 84.63it/s]


In [20]:
import numpy as np
import libpysal
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from spreg import OLS
from sklearn.preprocessing import StandardScaler

#2.构建ols模型
pop_X_modify=['% with RQF2+ - aged 16-64','% with RQF4+ - aged 16-64',
    'Aged 25 to 34 years', 'Aged 35 to 49 years', 'Aged 65 years and over']
# (1) 准备变量
X_df = pop[pop_X_modify].astype(float)
X = sm.add_constant(X_df)
Y = np.array(pop['avg_ai_exposure'], dtype=float).reshape(-1, 1)
# Ridge 回归（自动交叉验证调参）
# 标准化
scaler = StandardScaler()
X_scaled_R = scaler.fit_transform(X)
X = np.array(X, dtype=float)
# 生成自变量名称列表
name_x = ['const'] + pop_X_modify

ols = OLS(
    name_ds="pop",
    y=Y,
    x=X,
    w=queen_w_pop,
    name_y="avg_ai_exposure",
    name_x=name_x,
    name_w=queen_w_pop,
    moran=True,
    spat_diag=True
)

print("============")
print("\nOLS 回归摘要")
print(ols.summary)




OLS 回归摘要
REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :         pop
Weights matrix      :<libpysal.weights.contiguity.Queen object at 0x000002249D9FB5C0>
Dependent Variable  :avg_ai_exposure                Number of Observations:         312
Mean dependent var  :      0.5053                Number of Variables   :           6
S.D. dependent var  :      0.0527                Degrees of Freedom    :         306
R-squared           :      0.6506
Adjusted R-squared  :      0.6449
Sum squared residual:    0.301441                F-statistic           :    113.9803
Sigma-square        :       0.001                Prob(F-statistic)     :   1.002e-67
S.E. of regression  :       0.031                Log likelihood        :     640.272
Sigma-square ML     :       0.001                Akaike info criterion :   -1268.544
S.E of regression ML:      0.0311                Schwarz criterion     :   -1246.

### 多重共线性与异方差检验

In [21]:
from statsmodels.stats.diagnostic import het_breuschpagan

# 计算 VIF 报告
# 确保 X_selected_array 有列名，转换为 DataFrame 以匹配
X_selected_df = pd.DataFrame(X, columns=name_x)
vif_data = pd.DataFrame()
vif_data["Variable"] = X_selected_df.columns
vif_data["VIF"] = [variance_inflation_factor(X_selected_df.values, i) 
for i in range(X_selected_df.shape[1])]
print("VIF 报告：")
print(vif_data.sort_values(by="VIF", ascending=False).to_string(index=False))

# 1. 首先获取模型的残差
residuals = ols.u  # pysal OLS模型的残差存储在u属性中
print(f"残差获取完成，长度：{len(residuals)}")
# 2. 准备解释变量矩阵（与原始模型相同）
exog = X
print(f"解释变量矩阵准备完成，形状：{exog.shape}")
# 3. 执行Breusch-Pagan检验
bp_test = het_breuschpagan(residuals, exog)
# 4. 输出结果
labels = ['LM Statistic', 'LM-Test p-value', 'F-Statistic', 'F-Test p-value']
bp_results = dict(zip(labels, bp_test))
print("Breusch-Pagan 检验结果：")
for label, value in bp_results.items():
    print(f"{label}: {value:.4f}")

VIF 报告：
                 Variable         VIF
                    const 1393.856292
   Aged 65 years and over    8.390290
      Aged 25 to 34 years    4.157430
      Aged 35 to 49 years    4.030010
% with RQF4+ - aged 16-64    2.330497
% with RQF2+ - aged 16-64    1.906173
残差获取完成，长度：312
解释变量矩阵准备完成，形状：(312, 6)
Breusch-Pagan 检验结果：
LM Statistic: 7.8603
LM-Test p-value: 0.1641
F-Statistic: 1.5817
F-Test p-value: 0.1649


### SEM空间误差模型

In [22]:
# 执行分析
pop_sem_test = sem_with_spatial_tests(
    X_df=pop[pop_X].astype(float),
    y=pop['avg_ai_exposure'],
    w=queen_w_pop,  # 确保已定义空间权重矩阵
    max_vars=9
)

1变量组合: 0it [00:00, ?it/s]f:\Python 3.13\Lib\site-packages\spreg\ml_error.py:184: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  res = minimize_scalar(
1变量组合: 1it [00:00,  5.85it/s]f:\Python 3.13\Lib\site-packages\spreg\ml_error.py:184: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  res = minimize_scalar(
1变量组合: 2it [00:00,  7.11it/s]f:\Python 3.13\Lib\site-packages\spreg\ml_error.py:184: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  res = minimize_scalar(
1变量组合: 3it [00:00,  5.76it/s]f:\Python 3.13\Lib\site-packages\spreg\ml_error.py:184: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  res = minimize_scalar(
1变量组合: 4it [00:00,  6.75it/s]f:\Python 3.13\Lib\site-packages\spreg\ml_error.py:184: RuntimeWarning: Method 'bounded' does not s

In [23]:
from spreg import ML_Error
# 3. 运行 sem 模型
sem = ML_Error(
    name_ds="occ",
    y=Y,
    x=X,
    w=queen_w_pop,
    name_y="avg_ai_exposure",
    name_x=name_x,
    name_w=queen_w_pop,
    robust='white',
    spat_diag=True,
    slx_lags=1,  # 生成 WX
)
print("=== SEM 模型结果 ===")
print(sem.summary)

print("\n=== 模型比较  ===")
print(f"OLS AIC : {ols.aic}")
print(f"SEM AIC : {sem.aic}")

f:\Python 3.13\Lib\site-packages\spreg\ml_error.py:184: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  res = minimize_scalar(


=== SEM 模型结果 ===
REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ML SPATIAL ERROR WITH SLX (SLX-Error) (METHOD = full)
------------------------------------------------------------------------
Data set            :         occ
Weights matrix      :<libpysal.weights.contiguity.Queen object at 0x000002249D9FB5C0>
Dependent Variable  :avg_ai_exposure                Number of Observations:         312
Mean dependent var  :      0.5053                Number of Variables   :          11
S.D. dependent var  :      0.0527                Degrees of Freedom    :         301
Pseudo R-squared    :      0.6787
Log likelihood      :    653.3113
Sigma-square ML     :      0.0009                Akaike info criterion :   -1284.623
S.E of regression   :      0.0298                Schwarz criterion     :   -1243.450

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------

### SAR空间滞后模型

In [24]:
# 执行分析
pop_sar_test = sar_with_spatial_tests(
    X_df=pop[pop_X].astype(float),
    y=pop['avg_ai_exposure'],
    w=queen_w_pop,  # 确保已定义空间权重矩阵
    max_vars=9
)

1变量组合: 9it [00:01,  5.89it/s]
2变量组合: 36it [00:05,  6.75it/s]
3变量组合: 84it [00:13,  6.45it/s]
4变量组合: 126it [00:17,  7.11it/s]
5变量组合: 126it [00:14,  8.70it/s]
6变量组合: 84it [00:07, 10.62it/s]
7变量组合: 36it [00:03,  9.24it/s]
8变量组合: 9it [00:01,  6.54it/s]
9变量组合: 1it [00:00,  7.28it/s]


In [25]:
from spreg import ML_Lag

sar = ML_Lag(
    name_ds="pop",
    y=Y,
    x=X,
    w=queen_w_pop,
    name_y="avg_ai_exposure",
    name_x=name_x,
    name_w=queen_w_pop,
    robust='white',
    spat_diag=True,
    slx_lags=1,  # 生成 WX
)
print("=== SAR 模型结果 ===")
print(sar.summary)

print("\n=== 模型比较  ===")
print(f"OLS AIC : {ols.aic}")
print(f"SAR AIC : {sar.aic}")



=== SAR 模型结果 ===
REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG WITH SLX - SPATIAL DURBIN MODEL (METHOD = FULL)
-------------------------------------------------------------------------------------------------
Data set            :         pop
Weights matrix      :<libpysal.weights.contiguity.Queen object at 0x000002249D9FB5C0>
Dependent Variable  :avg_ai_exposure                Number of Observations:         312
Mean dependent var  :      0.5053                Number of Variables   :          12
S.D. dependent var  :      0.0527                Degrees of Freedom    :         300
Pseudo R-squared    :      0.6790
Spatial Pseudo R-squared:  0.6790
Log likelihood      :    653.4207
Sigma-square ML     :      0.0009                Akaike info criterion :   -1282.841
S.E of regression   :      0.0298                Schwarz criterion     :   -1237.925

------------------------------------------------------------------------------------
            

### SDM空间杜宾模型

In [26]:
# 执行分析
pop_sdm_test = sdm_with_spatial_tests(
    X_df=pop[pop_X].astype(float),
    y=pop['avg_ai_exposure'],
    w=queen_w_pop,  # 确保已定义空间权重矩阵
    max_vars=9
)

1变量组合: 9it [00:00, 35.97it/s]
2变量组合: 36it [00:00, 59.29it/s]
3变量组合: 84it [00:01, 59.46it/s]
4变量组合: 126it [00:02, 55.99it/s]
5变量组合: 126it [00:02, 50.79it/s]
6变量组合: 84it [00:07, 11.02it/s]
7变量组合: 36it [00:01, 27.57it/s]
8变量组合: 9it [00:00, 36.38it/s]
9变量组合: 1it [00:00, 39.75it/s]


In [27]:
from spreg import GM_Combo

# 拟合SDM模型
sdm = GM_Combo(
    name_ds="pop",
    y=Y,
    x=X,
    w=queen_w_pop,
    name_y="avg_ai_exposure",
    name_x=name_x,
    name_w=queen_w_pop,
    slx_lags=1
)
print("=== SDM 模型结果 ===")
print(sdm.summary)


=== SDM 模型结果 ===
REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: SPATIALLY WEIGHTED 2SLS - GM-COMBO MODEL WITH SLX (GNSM)
---------------------------------------------------------------------------
Data set            :         pop
Weights matrix      :<libpysal.weights.contiguity.Queen object at 0x000002249D9FB5C0>
Dependent Variable  :avg_ai_exposure                Number of Observations:         312
Mean dependent var  :      0.5053                Number of Variables   :          12
S.D. dependent var  :      0.0527                Degrees of Freedom    :         300
Pseudo R-squared    :      0.6056
Spatial Pseudo R-squared: omitted due to rho outside the boundary (-1, 1).

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT         0.00054         0.05

## (3)产业指标

### 变量组合尝试

In [28]:
# 1.构建几何邻接关系
queen_w = libpysal.weights.Queen.from_dataframe(ind)  # 默认基于几何边界
queen_w.transform = 'r'  # 行标准化（推荐）
# 检查孤岛（无邻居的区域）
isolated = np.where(np.array([len(queen_w.neighbors[i]) for i in range(queen_w.n)]) == 0)[0]
print(f"孤岛数量：{len(isolated)}，索引：{isolated}")
# 剔除孤岛（可选）
if len(isolated) > 0:
    mask = np.ones(queen_w.n, dtype=bool)
    mask[isolated] = False
    ind = ind.iloc[mask]
    queen_w_ind = libpysal.weights.Queen.from_dataframe(ind)
else:
    queen_w_ind = queen_w    

queen_w_ind.transform = 'r'

C:\Users\uiol\AppData\Local\Temp\ipykernel_25232\503096415.py:2: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  queen_w = libpysal.weights.Queen.from_dataframe(ind)  # 默认基于几何边界
f:\Python 3.13\Lib\site-packages\libpysal\weights\contiguity.py:347: UserWarning: The weights matrix is not fully connected: 
 There are 4 disconnected components.
 There are 3 islands with ids: 43, 49, 294.
  W.__init__(self, neighbors, ids=ids, **kw)
C:\Users\uiol\AppData\Local\Temp\ipykernel_25232\503096415.py:12: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  queen_w_ind = libpysal.weights.Queen.from_dataframe(ind)


('WARNING: ', 43, ' is an island (no neighbors)')
('WARNING: ', 49, ' is an island (no neighbors)')
('WARNING: ', 294, ' is an island (no neighbors)')
孤岛数量：3，索引：[ 43  49 294]


### Queen&OLS建模

In [29]:

ind_ols_test = ols_with_spatial_tests(
    X_df=ind[ind_X].astype(float),
    y=ind['avg_ai_exposure'],
    w=queen_w_ind,  # 确保已定义空间权重矩阵
    max_vars=18
)

# 保存完整结果
#occ_X_test.to_excel('ols_with_spatial_tests.xlsx', index=False)

1变量组合: 18it [00:00, 47.64it/s]
2变量组合: 153it [00:02, 56.44it/s]
3变量组合: 816it [00:14, 56.44it/s]
4变量组合: 3060it [00:54, 56.45it/s]
5变量组合: 8568it [02:53, 49.28it/s]
6变量组合: 18564it [05:31, 56.01it/s]
7变量组合: 31824it [06:35, 80.51it/s]
8变量组合: 43758it [09:19, 78.22it/s]
9变量组合: 48620it [13:17, 60.97it/s]
10变量组合: 43758it [10:52, 67.08it/s]
11变量组合: 31824it [09:39, 54.92it/s]
12变量组合: 18564it [05:38, 54.83it/s]
13变量组合: 8568it [02:52, 49.66it/s]
14变量组合: 3060it [01:01, 50.08it/s]
15变量组合: 816it [00:19, 42.32it/s]
16变量组合: 153it [00:03, 40.89it/s]
17变量组合: 18it [00:00, 37.22it/s]
18变量组合: 1it [00:00, 29.12it/s]


In [30]:
import numpy as np
import libpysal
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from spreg import OLS
from sklearn.preprocessing import StandardScaler

#2.构建ols模型
# (1) 准备变量

ind_X_modify=[
    'F: Construction',
    'H: Transport and storage',
    'I: Accommodation and food service activities',
    'J: Information and communication',
    'K: Financial and insurance activities', 'L: Real estate activities',
    'M: Professional, scientific and technical activities',
    'O: Public administration and defence; compulsory social security',
    'P: Education', 'Q: Human health and social work activities',
    ]

X_df = ind[ind_X_modify].astype(float)
# 添加常数项（保持为 DataFrame）
X = sm.add_constant(X_df)
# 因变量
Y = np.array(ind['avg_ai_exposure'], dtype=float).reshape(-1, 1)
# Ridge 回归（自动交叉验证调参）
# 标准化
scaler = StandardScaler()
X_scaled_R = scaler.fit_transform(X)
# 转换为 NumPy 数组用于 OLS（仅在必要时）
X = np.array(X, dtype=float)
# 生成自变量名称列表
name_x = ['const'] + ind_X_modify
# 运行 OLS 回归
ols = OLS(
    name_ds="ind",
    y=Y,
    x=X,
    w=queen_w_ind,  # 必须提供权重矩阵
    name_y="avg_ai_exposure",  # 明确因变量名
    name_x=name_x,    # 使用筛选后的自变量名
    name_w=queen_w_ind,
    moran=True,
    spat_diag=True
)
# 打印 OLS 结果
print("============")
print("\nOLS 回归摘要")
print(ols.summary)




OLS 回归摘要
REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :         ind
Weights matrix      :<libpysal.weights.contiguity.Queen object at 0x00000224828F71D0>
Dependent Variable  :avg_ai_exposure                Number of Observations:         313
Mean dependent var  :      0.5056                Number of Variables   :          11
S.D. dependent var  :      0.0529                Degrees of Freedom    :         302
R-squared           :      0.6992
Adjusted R-squared  :      0.6892
Sum squared residual:    0.262846                F-statistic           :     70.1994
Sigma-square        :       0.001                Prob(F-statistic)     :   9.271e-73
S.E. of regression  :       0.030                Log likelihood        :     664.266
Sigma-square ML     :       0.001                Akaike info criterion :   -1306.533
S.E of regression ML:      0.0290                Schwarz criterion     :   -1265.

### SEM空间误差模型

In [37]:
#减少变量值C、F-Q
ind_X=[
    'C: Manufacturing','F: Construction',
    'G: Wholesale and retail trade; repair of motor vehicles and motorcycles',
    'H: Transport and storage','I: Accommodation and food service activities',
    'J: Information and communication','K: Financial and insurance activities', 'L: Real estate activities',
    'M: Professional, scientific and technical activities','N: Administrative and support service activities',
    'O: Public administration and defence; compulsory social security',
    'P: Education', 'Q: Human health and social work activities',
    ]


In [38]:

ind_sem_test = sem_with_spatial_tests(
    X_df=ind[ind_X].astype(float),
    y=ind['avg_ai_exposure'],
    w=queen_w_ind,  # 确保已定义空间权重矩阵
    max_vars=13
)

# 保存完整结果
#occ_X_test.to_excel('ols_with_spatial_tests.xlsx', index=False)

1变量组合: 0it [00:00, ?it/s]f:\Python 3.13\Lib\site-packages\spreg\ml_error.py:184: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  res = minimize_scalar(
1变量组合: 1it [00:00,  4.13it/s]f:\Python 3.13\Lib\site-packages\spreg\ml_error.py:184: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  res = minimize_scalar(
1变量组合: 2it [00:00,  2.75it/s]f:\Python 3.13\Lib\site-packages\spreg\ml_error.py:184: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  res = minimize_scalar(
1变量组合: 3it [00:00,  4.07it/s]f:\Python 3.13\Lib\site-packages\spreg\ml_error.py:184: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  res = minimize_scalar(
1变量组合: 4it [00:00,  4.90it/s]f:\Python 3.13\Lib\site-packages\spreg\ml_error.py:184: RuntimeWarning: Method 'bounded' does not s

In [ ]:
from spreg import ML_Error
# 3. 运行 sem 模型
sem = ML_Error(
    name_ds="occ",
    y=Y,
    x=X,
    w=queen_w_ind,
    name_y="avg_ai_exposure",
    name_x=name_x,
    name_w=queen_w_ind,
    robust='white',
    spat_diag=True,
    slx_lags=1,  # 生成 WX
)
print("=== SEM 模型结果 ===")
print(sem.summary)

print("\n=== 模型比较  ===")
print(f"OLS AIC : {ols.aic}")
print(f"SEM AIC : {sem.aic}")

存在空间滞后依赖（lag显著），建议转向空间滞后模型（SLM）或SDM

### 多重共线性与异方差检验

In [ ]:
from statsmodels.stats.diagnostic import het_breuschpagan

# 计算 VIF 报告
# 确保 X_selected_array 有列名，转换为 DataFrame 以匹配
X_selected_df = pd.DataFrame(X, columns=name_x)
vif_data = pd.DataFrame()
vif_data["Variable"] = X_selected_df.columns
vif_data["VIF"] = [variance_inflation_factor(X_selected_df.values, i) 
for i in range(X_selected_df.shape[1])]
print("VIF 报告：")
print(vif_data.sort_values(by="VIF", ascending=False).to_string(index=False))

# 1. 首先获取模型的残差
residuals = ols.u  # pysal OLS模型的残差存储在u属性中
print(f"残差获取完成，长度：{len(residuals)}")
# 2. 准备解释变量矩阵（与原始模型相同）
exog = X
print(f"解释变量矩阵准备完成，形状：{exog.shape}")
# 3. 执行Breusch-Pagan检验
bp_test = het_breuschpagan(residuals, exog)
# 4. 输出结果
labels = ['LM Statistic', 'LM-Test p-value', 'F-Statistic', 'F-Test p-value']
bp_results = dict(zip(labels, bp_test))
print("Breusch-Pagan 检验结果：")
for label, value in bp_results.items():
    print(f"{label}: {value:.4f}")

严重的多重共线性，建议使用主成分分析、空间PCA、岭回归

### 使用SAR空间滞后模型

In [39]:

ind_sar_test = sar_with_spatial_tests(
    X_df=ind[ind_X].astype(float),
    y=ind['avg_ai_exposure'],
    w=queen_w_ind,  # 确保已定义空间权重矩阵
    max_vars=18
)

# 保存完整结果
#occ_X_test.to_excel('ols_with_spatial_tests.xlsx', index=False)

1变量组合: 13it [00:02,  5.13it/s]
2变量组合: 78it [00:12,  6.04it/s]
3变量组合: 286it [00:37,  7.59it/s]
4变量组合: 715it [01:42,  6.98it/s]
5变量组合: 1287it [03:17,  6.52it/s]
6变量组合: 1716it [04:08,  6.92it/s]
7变量组合: 1716it [03:58,  7.19it/s]
8变量组合: 1287it [02:58,  7.20it/s]
9变量组合: 715it [01:40,  7.11it/s]
10变量组合: 286it [00:39,  7.19it/s]
11变量组合: 78it [00:10,  7.12it/s]
12变量组合: 13it [00:01,  8.52it/s]
13变量组合: 1it [00:00, 10.86it/s]


In [ ]:
from spreg import ML_Lag

sar = ML_Lag(
    name_ds="ind",
    y=Y,
    x=X,
    w=queen_w_ind,
    name_y="avg_ai_exposure",
    name_x=name_x,
    name_w=queen_w_ind,
    robust='white',
    slx_lags=1,  # 生成 WX
)
print("=== SAR 模型结果 ===")
print(sar.summary)

print("\n=== 模型比较  ===")
print(f"OLS AIC : {ols.aic}")
print(f"SAR AIC : {sar.aic}")

### 使用SDM空间杜宾模型

In [40]:

ind_sdm_test = sdm_with_spatial_tests(
    X_df=ind[ind_X].astype(float),
    y=ind['avg_ai_exposure'],
    w=queen_w_ind,  # 确保已定义空间权重矩阵
    max_vars=18
)

# 保存完整结果
#occ_X_test.to_excel('ols_with_spatial_tests.xlsx', index=False)

1变量组合: 13it [00:00, 47.69it/s]
2变量组合: 78it [00:01, 60.17it/s]
3变量组合: 286it [00:04, 57.81it/s]
4变量组合: 715it [00:14, 48.84it/s]
5变量组合: 1287it [00:26, 48.46it/s]
6变量组合: 1548it [00:32, 37.04it/s]

组合 ('H: Transport and storage', 'I: Accommodation and food service activities', 'K: Financial and insurance activities', 'L: Real estate activities', 'M: Professional, scientific and technical activities', 'N: Administrative and support service activities') 建模失败: power expansion will not converge, check model specification and that weight are less than 1


6变量组合: 1716it [00:39, 43.32it/s]
7变量组合: 1716it [00:33, 51.37it/s]
8变量组合: 1287it [00:28, 44.72it/s]
9变量组合: 715it [00:16, 43.95it/s]
10变量组合: 286it [00:06, 41.25it/s]
11变量组合: 78it [00:01, 43.54it/s]
12变量组合: 13it [00:00, 47.98it/s]
13变量组合: 1it [00:00, 44.75it/s]


In [ ]:
from spreg import GM_Combo

# 拟合SDM模型
sdm = GM_Combo(
    name_ds="ind",
    y=Y,
    x=X,
    w=queen_w_ind,
    name_y="avg_ai_exposure",
    name_x=name_x,
    name_w=queen_w_ind,
    slx_lags=1
)
print("=== SDM 模型结果 ===")
print(sdm.summary)


## 输出所有模拟组合

In [45]:
import pandas as pd

# 定义输出函数
def save_top20_to_excel(df_dict, categories):
    """
    将多个数据框按AIC排序后输出前20到Excel
    参数:
        df_dict: 包含所有数据框的字典
        prefix_list: 前缀列表['ind', 'pop', 'occ']
    """
    for category in categories:
        # 创建Excel写入对象
        output_path = f"{category}_top_models.xlsx"
        writer = pd.ExcelWriter(output_path, engine='xlsxwriter')
        
        # 处理sdm结果
        sdm_key = f"{category}_sdm_test"
        if sdm_key in df_dict and df_dict[sdm_key] is not None:
            sdm_df = df_dict[sdm_key].sort_values(by='R²', ascending=False).head(20)  # 添加ascending=False
            sdm_df.to_excel(writer, sheet_name=f"{category}_SDM", index=False)
        
        # 处理sem结果
        sem_key = f"{category}_sem_test"
        if sem_key in df_dict and df_dict[sem_key] is not None:
            sem_df = df_dict[sem_key].sort_values(by='AIC').head(20)
            sem_df.to_excel(writer, sheet_name=f"{category}_SEM", index=False)

        # 处理sar结果
        sar_key = f"{category}_sar_test"
        if sar_key in df_dict and df_dict[sar_key] is not None:
            sar_df = df_dict[sar_key].sort_values(by='AIC').head(20)
            sar_df.to_excel(writer, sheet_name=f"{category}_SAR", index=False)

        # 处理OLS结果
        ols_key = f"{category}_ols_test"
        if ols_key in df_dict and df_dict[ols_key] is not None:
            ols_df = df_dict[ols_key].sort_values(by='AIC').head(20)
            ols_df.to_excel(writer, sheet_name=f"{category}_OLS", index=False)
        
        # 保存Excel文件
        writer.close()
        print(f"已保存 {category} 结果到 {output_path}")

# 假设已有以下数据框（实际使用时请取消注释）
dataframes = {
    'ind_sem_test': ind_sem_test,
    'ind_sar_test': ind_sar_test,
    'ind_sdm_test': ind_sdm_test,
    'ind_ols_test': ind_ols_test,
    'pop_sem_test': pop_sem_test,
    'pop_sar_test': pop_sar_test,
    'pop_sdm_test': pop_sdm_test,
    'pop_ols_test': pop_ols_test,
    'occ_sem_test': occ_sem_test,
    'occ_sar_test': occ_sar_test,
    'occ_sdm_test': occ_sdm_test,
    'occ_ols_test': occ_ols_test
}

# 执行输出
save_top20_to_excel(dataframes, ['ind', 'pop', 'occ'])

已保存 ind 结果到 ind_top_models.xlsx
已保存 pop 结果到 pop_top_models.xlsx
已保存 occ 结果到 occ_top_models.xlsx


**尝试组合数**
ind_sem_test :8191
ind_sar_test :8191
ind_sdm_test :8190
ind_ols_test :8190
pop_sem_test :511
pop_sar_test :511
pop_sdm_test :511
pop_ols_test :511
occ_sem_test :127
occ_sar_test :127
occ_sdm_test :127
occ_ols_test :127
